In [1]:
%matplotlib widget

In [2]:
import pandas as pd
import numpy as np
from glob import glob
import os
import datetime
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Input, Concatenate, Dot, Add, ReLU, Activation
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow import keras

In [3]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [4]:
folder = 'data'
file_names = [['가평_2016.xlsx','가평_2017.xlsx','가평_2018.xlsx', '가평_2019.xlsx'], 
              ['의암호_2016.xlsx','의암호_2017.xlsx','의암호_2018.xlsx', '의암호_2019.xlsx'],
              ['서상_2016.xlsx','서상_2017.xlsx','서상_2018.xlsx', '서상_2019.xlsx']]

# file_names = [['가평_2016.xlsx','가평_2017.xlsx','가평_2018.xlsx', '가평_2019.xlsx'], 
#               ['의암호_2016.xlsx','의암호_2017.xlsx','의암호_2018.xlsx', '의암호_2019.xlsx']]
day = 24*60*60
year = (365.2425)*day

df_full = []
df = []

for loc in range(len(file_names)):
    
    df_loc = []
    for y in range(len(file_names[loc])):
        path = os.path.join(folder, file_names[loc][y])
        print(file_names[loc][y])
        df_loc.append(pd.read_excel(path))
    df_full.append(pd.concat(df_loc))
    df.append(df_full[loc].iloc[:,2:])
#     df.append(df_full[loc].iloc[:, [2,3,4,5,6,7,10]])
#     df.append(df_full[loc].iloc[:, 2:11])
    date_time = pd.to_datetime(df_full[loc].iloc[:, 0], format='%Y.%m.%d %H:%M', utc=True)
    timestamp_s = date_time.map(datetime.datetime.timestamp)
    df[loc]['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
    df[loc]['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
    df[loc]['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
    df[loc]['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))
    df[loc] = df[loc].reset_index(drop=True)
        

가평_2016.xlsx
가평_2017.xlsx
가평_2018.xlsx
가평_2019.xlsx
의암호_2016.xlsx
의암호_2017.xlsx
의암호_2018.xlsx
의암호_2019.xlsx
서상_2016.xlsx
서상_2017.xlsx
서상_2018.xlsx
서상_2019.xlsx


In [5]:
df[0]

,수온,수소이온농도,전기전도도,용존산소,탁도,총유기탄소,총질소,총인,클로로필-a,미생물_독성지수,Day sin,Day cos,Year sin,Year cos
0,4.5,7.5,126.0,10.8,0.6,1.4,2.398,0.006,5.7,0.5,-1.407890e-12,1.000000,-0.002666,0.999996
1,4.3,7.5,126.0,10.8,0.5,1.4,2.490,0.003,5.4,0.4,2.588190e-01,0.965926,-0.001950,0.999998
2,4.4,7.5,126.0,10.8,0.5,1.4,2.545,0.003,5.4,0.5,5.000000e-01,0.866025,-0.001233,0.999999
3,4.4,7.5,126.0,10.8,0.6,1.4,2.482,0.002,5.4,4.7,7.071068e-01,0.707107,-0.000516,1.000000
4,4.4,7.5,126.0,10.8,0.6,1.4,2.453,0.003,5.4,-2.5,8.660254e-01,0.500000,0.000201,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,3.6,7.1,115.0,12.6,1.1,1.8,1.718,0.002,22.7,0.1,-9.659258e-01,0.258819,-0.005734,0.999984
35060,3.5,7.1,115.0,12.6,1.1,1.7,1.726,0.001,21.9,0.5,-8.660254e-01,0.500000,-0.005017,0.999987
35061,3.4,7.1,115.0,12.6,1.1,1.7,1.738,0.002,21.3,0.9,-7.071068e-01,0.707107,-0.004301,0.999991
35062,3.4,7.1,115.0,12.6,1.1,1.7,1.723,0.002,21.5,0.7,-5.000000e-01,0.866025,-0.003584,0.999994


In [6]:
df[0]

,수온,수소이온농도,전기전도도,용존산소,탁도,총유기탄소,총질소,총인,클로로필-a,미생물_독성지수,Day sin,Day cos,Year sin,Year cos
0,4.5,7.5,126.0,10.8,0.6,1.4,2.398,0.006,5.7,0.5,-1.407890e-12,1.000000,-0.002666,0.999996
1,4.3,7.5,126.0,10.8,0.5,1.4,2.490,0.003,5.4,0.4,2.588190e-01,0.965926,-0.001950,0.999998
2,4.4,7.5,126.0,10.8,0.5,1.4,2.545,0.003,5.4,0.5,5.000000e-01,0.866025,-0.001233,0.999999
3,4.4,7.5,126.0,10.8,0.6,1.4,2.482,0.002,5.4,4.7,7.071068e-01,0.707107,-0.000516,1.000000
4,4.4,7.5,126.0,10.8,0.6,1.4,2.453,0.003,5.4,-2.5,8.660254e-01,0.500000,0.000201,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,3.6,7.1,115.0,12.6,1.1,1.8,1.718,0.002,22.7,0.1,-9.659258e-01,0.258819,-0.005734,0.999984
35060,3.5,7.1,115.0,12.6,1.1,1.7,1.726,0.001,21.9,0.5,-8.660254e-01,0.500000,-0.005017,0.999987
35061,3.4,7.1,115.0,12.6,1.1,1.7,1.738,0.002,21.3,0.9,-7.071068e-01,0.707107,-0.004301,0.999991
35062,3.4,7.1,115.0,12.6,1.1,1.7,1.723,0.002,21.5,0.7,-5.000000e-01,0.866025,-0.003584,0.999994


In [7]:
# normalize data

df_all = pd.concat(df)
df_all

train_mean = df_all.mean()
train_std = df_all.std()
for i in range(len(file_names)):
    df[i] = (df[i]-train_mean)/train_std

In [8]:
train_mean, train_std

(수온          1.371826e+01
 수소이온농도      7.363691e+00
 전기전도도       1.062274e+02
 용존산소        1.048823e+01
 탁도          2.190726e+00
 총유기탄소       1.599534e+00
 총질소         1.617681e+00
 총인          8.346780e-03
 클로로필-a      8.673920e+00
 미생물_독성지수   -4.499445e-02
 Day sin     6.037103e-15
 Day cos     2.747682e-15
 Year sin   -5.681266e-08
 Year cos    2.053380e-05
 dtype: float64,
 수온           7.365852
 수소이온농도       0.482657
 전기전도도       13.561701
 용존산소         2.071821
 탁도           8.264139
 총유기탄소        0.357371
 총질소          0.398078
 총인           0.007764
 클로로필-a       9.531307
 미생물_독성지수     1.615261
 Day sin      0.707110
 Day cos      0.707110
 Year sin     0.707103
 Year cos     0.707117
 dtype: float64)

In [9]:
df[0]

,수온,수소이온농도,전기전도도,용존산소,탁도,총유기탄소,총질소,총인,클로로필-a,미생물_독성지수,Day sin,Day cos,Year sin,Year cos
0,-1.251486,0.282415,1.457973,0.150479,-0.192485,-0.558340,1.960215,-0.302267,-0.312016,0.337403,-1.999586e-12,1.414207,-0.003771,1.414158
1,-1.278638,0.282415,1.457973,0.150479,-0.204586,-0.558340,2.191326,-0.688669,-0.343491,0.275494,3.660237e-01,1.366019,-0.002757,1.414161
2,-1.265062,0.282415,1.457973,0.150479,-0.204586,-0.558340,2.329489,-0.688669,-0.343491,0.337403,7.071034e-01,1.224739,-0.001743,1.414162
3,-1.265062,0.282415,1.457973,0.150479,-0.192485,-0.558340,2.171229,-0.817470,-0.343491,2.937603,9.999952e-01,0.999995,-0.000730,1.414163
4,-1.265062,0.282415,1.457973,0.150479,-0.192485,-0.558340,2.098379,-0.688669,-0.343491,-1.519882,1.224739e+00,0.707103,0.000284,1.414163
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,-1.373671,-0.546331,0.646865,1.019280,-0.131983,0.560945,0.252009,-0.817470,1.471580,0.089765,-1.366019e+00,0.366024,-0.008109,1.414140
35060,-1.387248,-0.546331,0.646865,1.019280,-0.131983,0.281124,0.272106,-0.946270,1.387646,0.337403,-1.224739e+00,0.707103,-0.007096,1.414145
35061,-1.400824,-0.546331,0.646865,1.019280,-0.131983,0.281124,0.302251,-0.817470,1.324696,0.585041,-9.999952e-01,0.999995,-0.006082,1.414150
35062,-1.400824,-0.546331,0.646865,1.019280,-0.131983,0.281124,0.264570,-0.817470,1.345679,0.461222,-7.071034e-01,1.224739,-0.005068,1.414154


In [10]:
train_df = df[0]
val_df = df[0]
test_df = df[0]

In [11]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df=train_df, val_df=val_df, test_df=test_df,
            #train_df=None, val_df=None, test_df=None,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

In [12]:
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

In [13]:
import matplotlib
import matplotlib.font_manager as fm
fm.get_fontconfig_fonts()
font_location = '/usr/share/fonts/truetype/nanum/NanumGothicCoding.ttf'
#font_location = '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc'
# font_location = 'C:/Windows/Fonts/NanumGothic.ttf' # For Windows
fprop = fm.FontProperties(fname=font_location)

In [14]:
def plot(self, model=None, plot_col='T (degC)', max_subplots=3):
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]', fontproperties=fprop)
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
    else:
      label_col_index = plot_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_col_index],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                  marker='X', edgecolors='k', label='Predictions',
                  c='#ff7f0e', s=64)

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot = plot

In [15]:
# not used
# original make_dataset code
def make_dataset(self, data):
  data = np.array(data, dtype=np.float32)
  ds = tf.keras.preprocessing.timeseries_dataset_from_array(
      data=data,
      targets=None,
      sequence_length=self.total_window_size,
      sequence_stride=1,
      shuffle=True,
      batch_size=32,)

  ds = ds.map(self.split_window)

  return ds

#WindowGenerator.make_dataset = make_dataset

In [16]:
@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example

In [17]:
def sample_batch_index(total, batch_size):
    '''Sample index of the mini-batch.

    Args:
        - total: total number of samples
        - batch_size: batch size

    Returns:
        - batch_idx: batch index
    '''
    total_idx = np.random.permutation(total)
    batch_idx = total_idx[:batch_size]
    return batch_idx

In [18]:
def binary_sampler(p, shape):
  '''Sample binary random variables.
  
  Args:
    - p: probability of 1
    - shape: matrix shape
    
  Returns:
    - binary_random_matrix: generated binary random matrix.
  '''
  unif_random_matrix = np.random.uniform(0., 1., size = shape)
  binary_random_matrix = 1*(unif_random_matrix < p)
  return binary_random_matrix

In [19]:
def uniform_sampler(low, high, shape):
  '''Sample uniform random variables.
  
  Args:
    - low: low limit
    - high: high limit
    - rows: the number of rows
    - cols: the number of columns
    
  Returns:
    - uniform_random_matrix: generated uniform random matrix.
  '''
  return np.random.uniform(low, high, size = shape)

In [20]:
def normalization (data, parameters=None):
  '''Normalize data in [0, 1] range.
  
  Args:
    - data: original data
  
  Returns:
    - norm_data: normalized data
    - norm_parameters: min_val, max_val for each feature for renormalization
  '''

  # Parameters
  _, dim = data.shape
  norm_data = data.copy()

  if parameters is None:

    # MixMax normalization
    min_val = np.zeros(dim)
    max_val = np.zeros(dim)
   
    # For each dimension
    for i in range(dim):
      min_val[i] = np.nanmin(norm_data[:,i])
      norm_data[:,i] = norm_data[:,i] - np.nanmin(norm_data[:,i])
      max_val[i] = np.nanmax(norm_data[:,i])
      norm_data[:,i] = norm_data[:,i] / (np.nanmax(norm_data[:,i]) + 1e-6)

    # Return norm_parameters for renormalization
    norm_parameters = {'min_val': min_val,
                       'max_val': max_val}
  else:
    min_val = parameters['min_val']
    max_val = parameters['max_val']

    # For each dimension
    for i in range(dim):
      norm_data[:,i] = norm_data[:,i] - min_val[i]
      norm_data[:,i] = norm_data[:,i] / (max_val[i] + 1e-6)

    norm_parameters = parameters

  return norm_data, norm_parameters

In [21]:
class MissData(object):
    def __init__(self, load_dir=None):
        if load_dir:
            self.missarr = np.load(os.path.join(load_dir, 'miss.npy'))
            self.idxarr = np.load(os.path.join(load_dir, 'idx.npy'))
            
    def make_missdata(self, data_x, missrate=0.2):
        data = data_x.copy()
        rows, cols = data_x.shape
        total_no = rows*cols
        total_miss_no = np.round(total_no*missrate).astype(int)
        total_idx = self.idxarr.shape[0]
        idxarr = self.idxarr
        missarr = self.missarr
        #print(total_miss_no)
        miss_no = 0
        cum_no = self.idxarr[:,3:4]
        cum_no = cum_no.reshape((total_idx))
        cum_sum = np.max(cum_no)
        #print(cum_no)
        #print(total_idx)
        while True:
            loc_count = np.around(np.random.random()*cum_sum)
            #print('loc_count =', loc_count)
            idx = len(cum_no[cum_no <= loc_count])-1
            #print(cum_no[cum_no <= loc_count])
            #print('idx =', idx)
            startnan = idxarr[idx][0]
            nanlen = idxarr[idx][2]
            loc = np.around(np.random.random()*(rows-nanlen)).astype(int)
            #print('loc =', loc)
            #print(loc_count, idx)
            #print(idxarr[idx])
            #data_copy = data[loc:loc+nanlen].copy()
            data_copy = data[loc:loc+nanlen]
            #print('startnan=', startnan)
            #isnan = missarr[startnan:startnan+nanlen].copy()
            isnan = missarr[startnan:startnan+nanlen]
            #print('isnan =',isnan)
            miss_no += idxarr[idx][1]
            if (miss_no > total_miss_no):
                break
            data_copy[isnan==1] = np.nan
            data[loc:loc+nanlen] = data_copy
        #print('miss_data =', data)
        return data
    
    def save(data, max_tseq, save_dir='save'):
        no, dim = data.shape
        #print((no, dim))
        isnan = np.isnan(data).astype(int)
        isany = np.any(isnan, axis=1).astype(int)
        shifted = np.roll(isany, 1)
        shifted[0] = 1
        #print(isnan)
        #print(isany.astype(int))
        #print(shifted)
        startnan = ((isany == 1) & (shifted ==0)).astype(int)
        #print(startnan)
        group = startnan.cumsum()
        group = group*isany
        #print(group)
        n = np.max(group)
        #print(n)
        missarr = None
        cum_no = 0
        rowidx = 0
        for i in range(1, n+1):
            g = (group == i).astype(int)
            i = np.argmax(g)
            rows = g.sum()
            #print(len)
            #print(i)
            #print(type(missarr))
            if rows <= max_tseq:
                nanseq = isnan[i:i+rows, :]
                no = np.sum(nanseq)
                #print(no)
                if missarr is None:
                    missarr = nanseq
                    idxarr = np.array([[rowidx, no, rows, cum_no]])
                else:
                    missarr = np.concatenate((missarr, nanseq))
                    idxarr = np.concatenate((idxarr, [[rowidx, no, rows, cum_no]]), axis=0)
                cum_no += no
                rowidx += rows

        #print(idxarr)
        miss_npy_file = os.path.join(save_dir, 'miss.npy')
        idx_npy_file = os.path.join(save_dir, 'idx.npy')
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        np.save(miss_npy_file, missarr)
        np.save(idx_npy_file, idxarr)
        print('miss_data file saved')

In [22]:
norm_df = pd.concat(df,axis=0)
norm_data = norm_df.to_numpy()
MissData.save(norm_data, max_tseq = 24)

miss_data file saved


In [23]:
def interpolate(np_data, max_gap=3):
    #n = np_data.shape[1]
    data = pd.DataFrame(np_data)
    #data[0][0] = np.nan
    #data[0][1] = np.nan
    #data[0][2] = np.nan
    #data[data.columns[0]][0] = np.nan
    #data[data.columns[0]][1] = np.nan
    #data[data.columns[0]][2] = np.nan
    
    # create mask
    mask = data.copy()
    grp = ((mask.notnull() != mask.shift().notnull()).cumsum())
    grp['ones'] = 1
    for i in data.columns:
        mask[i] = (grp.groupby(i)['ones'].transform('count') < max_gap) | data[i].notnull()
    data = data.interpolate(method='polynomial', order=5, limit=max_gap, axis=0).bfill()[mask]
    return data.to_numpy()
    #return data
    
#filled_data = interpolate(norm_data, max_gap=3)
#np.arange(0, 5, dtype=int)
#['%d'%val for val in range(0,5)]

In [24]:
class GainDataGenerator(keras.utils.Sequence):
    'Generates data for GAIN'
    def __init__(self,
                 data_list,
                 batch_size=32,
                 input_width=24*3,
                 label_width=24*3,
                 shift=0,
                 fill_no=4,
                 miss_rate=0.2,
                 hint_rate=0.9,
                 normalize=True,
                 miss_pattern=None,
                 alpha=100.):
        'Initialization'
        window_size = input_width
        
        # interpollation
        filled_data = []
        for data in data_list:
            data = interpolate(data, max_gap=fill_no)
            filled_data.append(data)
            
        data_list = filled_data
        
        # whole data
        self.data = np.concatenate(data_list)

        # TO-DO
        
        # pre calculation for  sequence data
        last_cum = 0
        cums = []
        for data in data_list:
            isnan = np.isnan(data)
            isany = np.any(isnan, axis=1)
            #shift same as pd.shift(isany, fill_value=True)
            shifted = np.roll(isany, 1)
            shifted[0] = True # set to nan
            
            start_seq = ((isany == False) & (shifted == True)).astype(int)
            cum = start_seq.cumsum()
            cum += last_cum
            last_cum = np.max(cum)
            cum[isany] = 0
            cums.append(cum)
            
        
        # normlize for spam
        if normalize:
            self.data, norm_param = normalization(self.data)
        #print(norm_param)
        
        # Define mask matrix
        if miss_pattern is None:
            self.data_m = binary_sampler(1-miss_rate, self.data.shape)
        else:
            #MissData.save(self.data, max_tseq = 12)
            self.miss = MissData(load_dir='save')
            self.miss_rate = miss_rate
            miss_data = self.miss.make_missdata(self.data, self.miss_rate)
            self.data_m = 1. - np.isnan(miss_data).astype(float)
            
            self.data_m_rand = binary_sampler(1-(miss_rate/10.), self.data.shape)
            self.data_m[self.data_m_rand==0.] = 0.
        self.miss_pattern = miss_pattern
        
        # sequence data
        self.ids = np.concatenate(cums)
        data_idx = np.empty((0), dtype=int)
        for i in range(1, last_cum+1):
            seq_len = (self.ids == i).sum()
            start_id = np.argmax(self.ids == i)
            # possible data number in seqeunce
            time_len = seq_len - window_size + 1
            start_ids = np.arange(start_id, start_id+time_len)
            data_idx = np.append(data_idx, start_ids)
            
        # start index set for sequence data
        self.data_idx = data_idx
        self.input_width = input_width
        self.no = len(data_idx)
        
        #print('self.no = ', self.no)
        
        self.batch_size = batch_size
        
        # random shuffling  index
        self.batch_idx = sample_batch_index(self.no, self.no)
        self.batch_id = 0
        self.shape = (batch_size,self.input_width)+self.data.shape[1:]
        #self.hint_rate = hint_rate
            
    def __len__(self):
        'Denotes the number of batches per epoch'
        #return int(128/self.batch_size)
        #return 2
        return 1

    def __getitem__(self, index):
        'Generate one batch of data'
        #print('index =', index)
        # Sample batch
        x = np.empty((0, self.input_width, self.data.shape[1]))
        #m = np.empty((0, self.input_width, self.data.shape[1]))
        #h = np.empty((0, self.input_width, self.data.shape[1]))
        y = np.empty((0, self.input_width, self.data.shape[1]))
        #print(x.shape)
        #print(self.data.shape)
        #print(self.input_width)
        #self.batch_idx = sample_batch_index(self.no, self.batch_size)
        for cnt in range(0, self.batch_size):
            i = self.batch_idx[self.batch_id]
            self.batch_id += 1
            #self.batch_id %= self.batch_size
            self.batch_id %= self.no
            if self.miss_pattern and (self.batch_id == 0):
                self.batch_idx = sample_batch_index(self.no, self.no)
                miss_data = self.miss.make_missdata(self.data, self.miss_rate)
                self.data_m = 1. - np.isnan(miss_data).astype(float)
                self.data_m_rand = binary_sampler(1-self.miss_rate/10., self.data.shape)
                self.data_m[self.data_m_rand==0.] = 0.
            idx1 = self.data_idx[i]
            idx2 = self.data_idx[i]+self.input_width
            #print(idx1, idx2)
        
            Y_mb = self.data[idx1:idx2].copy()
            X_mb = Y_mb.copy()
            M_mb = self.data_m[idx1:idx2]
            Z_mb = uniform_sampler(0, 0.01, shape=X_mb.shape)
            X_mb = M_mb*X_mb + (1-M_mb)*Z_mb
            #H_mb_temp = binary_sampler(self.hint_rate, shape=X_mb.shape)
            #H_mb = M_mb * H_mb_temp
            X_mb[M_mb == 0] = np.nan
            Y_mb[M_mb == 1] = np.nan
            x = np.append(x, [X_mb], axis=0)
            #m = np.append(m, [M_mb], axis=0)
            #h = np.append(h, [H_mb], axis=0)
            y = np.append(y, [Y_mb], axis=0)
            
        #return [x, m, h], y
        return x, y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        return

dgen = GainDataGenerator(df)

In [25]:
class GAIN(keras.Model):
    def __init__(self, shape, alpha=100., load=False, hint_rate=0.9, gen_sigmoid=True, **kwargs):
        super(GAIN, self).__init__(**kwargs)
        self.shape = shape
        self.dim = np.prod(shape).astype(int)
        self.h_dim = self.dim
        self.gen_sigmoid = gen_sigmoid
        self.build_generator()
        self.build_discriminator()
        self.hint_rate = hint_rate
        self.alpha = alpha
        self.generator_optimizer = Adam()
        self.discriminator_optimizer = Adam()

    ## GAIN models
    def build_generator(self):
        last_activation = 'sigmoid' if self.gen_sigmoid else None
        xavier_initializer = tf.keras.initializers.GlorotNormal()

        shape = self.shape
        #x = Input(shape=(self.dim,), name='generator_input_x')
        #m = Input(shape=(self.dim,), name='generator_input_m')
        x = Input(shape=shape, name='generator_input_x')
        m = Input(shape=shape, name='generator_input_m')
        
        x_f = Flatten()(x)
        m_f = Flatten()(m)

        a = Concatenate()([x_f, m_f])

        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        #a = keras.layers.BatchNormalization()(a)
        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        #a = keras.layers.BatchNormalization()(a)
        a = Dense(self.dim, activation=last_activation, kernel_initializer=xavier_initializer)(a)
        G_prob = keras.layers.Reshape(shape)(a)
        self.generator = keras.models.Model([x, m], G_prob, name='generator')

    def build_discriminator(self):
        xavier_initializer = tf.keras.initializers.GlorotNormal()
        shape = self.shape

        #x = Input(shape=(self.dim,), name='discriminator_input_x')
        #h = Input(shape=(self.dim,), name='discriminator_input_h')
        x = Input(shape=shape, name='discriminator_input_x')
        h = Input(shape=shape, name='discriminator_input_h')
        
        x_f = Flatten()(x)
        h_f = Flatten()(h)

        a = Concatenate()([x_f, h_f])

        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        a = Dense(self.dim, activation='sigmoid', kernel_initializer=xavier_initializer)(a)
        D_prob = keras.layers.Reshape(shape)(a)
        self.discriminator = keras.models.Model([x, h], D_prob, name='discriminator')
        
    def call(self, inputs):
        if isinstance(inputs, tuple):
            inputs = inputs[0]
        shape = inputs.shape
        dims = np.prod(shape[1:])
        input_width = shape[1]
        #print('inputs.shape=',inputs.shape)
        x = inputs
        #x = x.reshape((n, -1))
        #print('dims=',dims)
        #x = keras.layers.Reshape((dims,))(x)
        #x = keras.layers.Reshape(tf.TensorShape((self.dim,)))(x)
        #print('x =', x)
        #print('x.shape = ', x.shape)
        #x = keras.layers.Reshape(tf.TensorShape([57]))(x)
        
        isnan = tf.math.is_nan(x)
        #m = 1.- keras.backend.cast(isnan, dtype=tf.float32)
        m = tf.where(isnan, 0., 1.)
        z = keras.backend.random_uniform(shape=tf.shape(x), minval=0.0, maxval=0.01)
        x = tf.where(isnan, z, x)
        #z = uniform_sampler(0, 0.01, shape=x.shape)
        #z = tf.keras.backend.random_uniform(shape=x.shape, minval=0.0, maxval=0.01)
        imputed_data = self.generator([x, m], training=False)
        #imputed_data = m*x + (1-m)*imputed_data
        #imputed_data = tf.where(isnan, imputed_data, np.nan)
        imputed_data = tf.where(isnan, imputed_data, x)
        #imputed_data = keras.layers.Reshape(shape[1:])(imputed_data)
        #print('imputed_data.shape = ', imputed_data.shape)
        
        return imputed_data
    
    def D_loss(M, D_prob):
        ## GAIN loss
        return -tf.reduce_mean(M * tf.keras.backend.log(D_prob + 1e-8) \
                         + (1-M) * tf.keras.backend.log(1. - D_prob + 1e-8))
    
    def G_loss(self, M, D_prob, X, G_sample):
        G_loss_temp = -tf.reduce_mean((1-M) * tf.keras.backend.log(D_prob + 1e-8))
        MSE_loss = tf.reduce_mean((M * X - M * G_sample)**2) / (tf.reduce_mean(M) + 1e-8)
        #G_loss_temp = GAIN.G_loss_bincross(M, D_prob)
        #MSE_loss = GAIN.MSE_loss(M, X, G_sample)
        G_loss = G_loss_temp + self.alpha * MSE_loss
        return G_loss
        
    def RMSE_loss(y_true, y_pred):
        isnan = tf.math.is_nan(y_true)
        M = tf.where(isnan, 1., 0.)
        return tf.sqrt(tf.reduce_sum(tf.where(isnan, 0., y_pred-y_true)**2)/tf.reduce_sum(1-M))
    
    def train_step(self, data):
        #[x, m, h], y = data
        x, y = data
        #X = keras.layers.Reshape((self.dim,), input_shape=self.shape)(x)
        #Y = keras.layers.Reshape((self.dim,), input_shape=self.shape)(y)
        #X = keras.layers.Flatten()(x)
        #Y = keras.layers.Flatten()(y)
        X = x
        Y = y
        #X = tf.reshape(x, shape=(x.shape[0], -1))
        #Y = tf.reshape(y, shape=(x.shape[0], -1))
        isnan = tf.math.is_nan(X)
        #M = 1 - keras.backend.cast(isnan, dtype=tf.float32)
        M = tf.where(isnan, 0., 1.)
        Z = keras.backend.random_uniform(shape=tf.shape(X), minval=0.0, maxval=0.01)
        #H_temp = binary_sampler(self.hint_rate, shape=X.shape)
        H_rand = keras.backend.random_uniform(shape=tf.shape(X), minval=0.0, maxval=1.)
        #H_temp = 1*keras.backend.cast((H_rand < self.hint_rate), dtype=tf.float32)
        H_temp = tf.where(H_rand < self.hint_rate, 1., 0.)
        
        H = M * H_temp
        #X = M * X + (1-M) * Z
        X = tf.where(isnan, Z, X)
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            G_sample = self.generator([X, M], training=True)

            # Combine with observed data
            #Hat_X = tf.where(isnan, G_sample, X)
            Hat_X = X * M + G_sample * (1-M)
            D_prob = self.discriminator([Hat_X, H], training=True)
            gen_loss = self.G_loss(M, D_prob, X, G_sample)
            disc_loss = tf.keras.backend.mean(tf.keras.losses.binary_crossentropy(M, D_prob))
            #disc_loss = GAIN.D_loss(M, D_prob)
            #disc_loss = GAIN.D_loss(M, D_prob)

        gradients_of_generator = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)

        self.generator_optimizer.apply_gradients(zip(gradients_of_generator, self.generator.trainable_variables))
        self.discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, self.discriminator.trainable_variables))
        
        rmse = tf.sqrt(tf.reduce_sum(tf.where(isnan, G_sample - Y, 0.)**2)/tf.reduce_sum(1-M))
        return {
                 'gen_loss':     gen_loss,
                 'disc_loss':    disc_loss,
                 'rmse':         rmse,
               }
    
    def save(self, save_dir='save'):
        if not os.path.exists(save_dir):
          os.makedirs(save_dir)
        disc_savefile = os.path.join(save_dir, 'discriminator.h5')
        gen_savefile = os.path.join(save_dir, 'generator.h5')
        self.discriminator.save_weights(disc_savefile)
        self.generator.save_weights(gen_savefile)

    def load(self, save_dir='save'):
        disc_savefile = os.path.join(save_dir, 'discriminator.h5')
        gen_savefile = os.path.join(save_dir, 'generator.h5')
        try:
          self.discriminator.load_weights(disc_savefile)
          self.generator.load_weights(gen_savefile)
          print('model weights loaded')
        except:
          print('model loadinng error')

In [26]:
def make_dataset_gain(self, data):
  dg = GainDataGenerator(
      df,
      input_width = self.input_width,
      label_width = self.label_width,
      batch_size = 128,
      normalize = False,
      miss_pattern = True,
      miss_rate = 0.15,
      fill_no = 3,
  )
  self.dg = dg
  ds = tf.data.Dataset.from_generator(
      lambda: dg,
      output_types=(tf.float32, tf.float32),
      output_shapes=(
        dg.shape,
        dg.shape
        #[batch_size, train_generator.dim],
        #[batch_size, train_generator.dim],
      )
  )
  return ds

WindowGenerator.make_dataset = make_dataset_gain

In [27]:
train_df = df_all
val_df = df_all
test_df = df_all

In [28]:
wide_window = WindowGenerator(
    input_width=24*5, label_width=24*5, shift=0,
    #label_columns=['T (degC)']
)

wide_window

Total window size: 120
Input indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]
Label indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 

In [29]:
wide_window.example[0].shape

TensorShape([128, 120, 14])

In [30]:
df[0]

,수온,수소이온농도,전기전도도,용존산소,탁도,총유기탄소,총질소,총인,클로로필-a,미생물_독성지수,Day sin,Day cos,Year sin,Year cos
0,-1.251486,0.282415,1.457973,0.150479,-0.192485,-0.558340,1.960215,-0.302267,-0.312016,0.337403,-1.999586e-12,1.414207,-0.003771,1.414158
1,-1.278638,0.282415,1.457973,0.150479,-0.204586,-0.558340,2.191326,-0.688669,-0.343491,0.275494,3.660237e-01,1.366019,-0.002757,1.414161
2,-1.265062,0.282415,1.457973,0.150479,-0.204586,-0.558340,2.329489,-0.688669,-0.343491,0.337403,7.071034e-01,1.224739,-0.001743,1.414162
3,-1.265062,0.282415,1.457973,0.150479,-0.192485,-0.558340,2.171229,-0.817470,-0.343491,2.937603,9.999952e-01,0.999995,-0.000730,1.414163
4,-1.265062,0.282415,1.457973,0.150479,-0.192485,-0.558340,2.098379,-0.688669,-0.343491,-1.519882,1.224739e+00,0.707103,0.000284,1.414163
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,-1.373671,-0.546331,0.646865,1.019280,-0.131983,0.560945,0.252009,-0.817470,1.471580,0.089765,-1.366019e+00,0.366024,-0.008109,1.414140
35060,-1.387248,-0.546331,0.646865,1.019280,-0.131983,0.281124,0.272106,-0.946270,1.387646,0.337403,-1.224739e+00,0.707103,-0.007096,1.414145
35061,-1.400824,-0.546331,0.646865,1.019280,-0.131983,0.281124,0.302251,-0.817470,1.324696,0.585041,-9.999952e-01,0.999995,-0.006082,1.414150
35062,-1.400824,-0.546331,0.646865,1.019280,-0.131983,0.281124,0.264570,-0.817470,1.345679,0.461222,-7.071034e-01,1.224739,-0.005068,1.414154


In [31]:
wide_window.plot(plot_col='총유기탄소')
print('make_dataset_gain: dg.no = ', wide_window.dg.no)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

make_dataset_gain: dg.no =  2173


In [32]:
plt.figure(figsize=(9,10))
isnan = np.isnan(norm_data).astype(int)
data = isnan
n = data.shape[0]
seq_len = n//8
for i in range(8):
    plt.subplot(181+i)
    plt.imshow(data[i*seq_len:(i+1)*seq_len, 0:7], aspect='auto')
    plt.yticks([])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [33]:
plt.figure(figsize=(9,10))
n = wide_window.dg.data_m.shape[0]
train = n//8
for i in range(8):
    plt.subplot(181+i)
    plt.imshow(wide_window.dg.data_m[i*train:(i+1)*train, 0:7], aspect='auto')
    plt.yticks([])
#plt.imshow(wide_window.dg.data[0:100])
#plt.imshow(wide_window.dg.data_m[800:900], aspect='auto')
#print(wide_window.dg.data[0:50])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 컴파일 및 학습_GAIN

In [34]:
val_performance = {}
performance = {}

In [35]:
gain = GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False)
gain.compile(loss=GAIN.RMSE_loss)

In [36]:
MAX_EPOCHS = 2000

def compile_and_fit(model, window, patience=10):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  #model.compile(loss=tf.losses.MeanSquaredError(),
                #optimizer=tf.optimizers.Adam(),
                #metrics=[tf.metrics.MeanAbsoluteError()])
  model.compile(loss=GAIN.RMSE_loss)

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

**모델 학습**

In [37]:
history = compile_and_fit(gain, wide_window, patience=MAX_EPOCHS//5)


val_performance['Gain'] = gain.evaluate(wide_window.val)
performance['Gain'] = gain.evaluate(wide_window.test, verbose=0)

gain.save(save_dir='save')

Epoch 1/2000
1/1 [==============================] - 1s 1s/step - gen_loss: 103.8505 - disc_loss: 0.7189 - rmse: 0.9560 - val_loss: 0.8837
Epoch 2/2000
1/1 [==============================] - 0s 131ms/step - gen_loss: 89.0294 - disc_loss: 0.5265 - rmse: 0.8573 - val_loss: 0.7518
Epoch 3/2000
1/1 [==============================] - 0s 108ms/step - gen_loss: 66.5960 - disc_loss: 0.5479 - rmse: 0.7264 - val_loss: 0.7048
Epoch 4/2000
1/1 [==============================] - 0s 127ms/step - gen_loss: 65.9865 - disc_loss: 0.4141 - rmse: 0.7067 - val_loss: 0.6659
Epoch 5/2000
1/1 [==============================] - 0s 141ms/step - gen_loss: 57.7310 - disc_loss: 0.4216 - rmse: 0.6873 - val_loss: 0.6589
Epoch 6/2000
1/1 [==============================] - 0s 142ms/step - gen_loss: 44.9760 - disc_loss: 0.4321 - rmse: 0.6180 - val_loss: 0.5975
Epoch 7/2000
1/1 [==============================] - 0s 146ms/step - gen_loss: 47.8945 - disc_loss: 0.4045 - rmse: 0.5973 - val_loss: 0.5871
Epoch 8/2000
1/1 [====

1/1 [==============================] - 0s 137ms/step - gen_loss: 8.1480 - disc_loss: 0.2947 - rmse: 0.4117 - val_loss: 0.3466
Epoch 60/2000
1/1 [==============================] - 0s 151ms/step - gen_loss: 9.0689 - disc_loss: 0.3017 - rmse: 0.3420 - val_loss: 0.3613
Epoch 61/2000
1/1 [==============================] - 0s 123ms/step - gen_loss: 8.3184 - disc_loss: 0.3054 - rmse: 0.5521 - val_loss: 0.3233
Epoch 62/2000
1/1 [==============================] - 0s 130ms/step - gen_loss: 8.6900 - disc_loss: 0.2942 - rmse: 0.4350 - val_loss: 0.3598
Epoch 63/2000
1/1 [==============================] - 0s 130ms/step - gen_loss: 9.5688 - disc_loss: 0.3019 - rmse: 0.5123 - val_loss: 0.3777
Epoch 64/2000
1/1 [==============================] - 0s 177ms/step - gen_loss: 8.3989 - disc_loss: 0.3062 - rmse: 0.4938 - val_loss: 0.3792
Epoch 65/2000
1/1 [==============================] - 0s 147ms/step - gen_loss: 7.6469 - disc_loss: 0.2893 - rmse: 0.4191 - val_loss: 0.3677
Epoch 66/2000
1/1 [===============

1/1 [==============================] - 0s 127ms/step - gen_loss: 8.3004 - disc_loss: 0.2499 - rmse: 0.3009 - val_loss: 0.3637
Epoch 118/2000
1/1 [==============================] - 0s 153ms/step - gen_loss: 9.3287 - disc_loss: 0.2499 - rmse: 0.2881 - val_loss: 0.4804
Epoch 119/2000
1/1 [==============================] - 0s 345ms/step - gen_loss: 10.0837 - disc_loss: 0.2514 - rmse: 0.3032 - val_loss: 0.4681
Epoch 120/2000
1/1 [==============================] - 0s 143ms/step - gen_loss: 8.2110 - disc_loss: 0.2397 - rmse: 0.2909 - val_loss: 0.4456
Epoch 121/2000
1/1 [==============================] - 0s 110ms/step - gen_loss: 9.9172 - disc_loss: 0.2427 - rmse: 0.2845 - val_loss: 0.5510
Epoch 122/2000
1/1 [==============================] - 0s 124ms/step - gen_loss: 13.5774 - disc_loss: 0.2451 - rmse: 0.3216 - val_loss: 0.6111
Epoch 123/2000
1/1 [==============================] - 0s 133ms/step - gen_loss: 8.5770 - disc_loss: 0.2388 - rmse: 0.3067 - val_loss: 0.6309
Epoch 124/2000
1/1 [======

1/1 [==============================] - 0s 106ms/step - gen_loss: 6.7446 - disc_loss: 0.2176 - rmse: 0.4102 - val_loss: 0.2640
Epoch 176/2000
1/1 [==============================] - 0s 113ms/step - gen_loss: 6.5229 - disc_loss: 0.2183 - rmse: 0.5454 - val_loss: 0.2821
Epoch 177/2000
1/1 [==============================] - 0s 139ms/step - gen_loss: 6.0309 - disc_loss: 0.2154 - rmse: 0.3850 - val_loss: 0.2804
Epoch 178/2000
1/1 [==============================] - 0s 119ms/step - gen_loss: 5.6926 - disc_loss: 0.2069 - rmse: 0.3851 - val_loss: 0.2926
Epoch 179/2000
1/1 [==============================] - 0s 162ms/step - gen_loss: 6.4059 - disc_loss: 0.2051 - rmse: 0.7258 - val_loss: 0.3017
Epoch 180/2000
1/1 [==============================] - 0s 179ms/step - gen_loss: 5.4823 - disc_loss: 0.2153 - rmse: 0.5485 - val_loss: 0.2890
Epoch 181/2000
1/1 [==============================] - 0s 132ms/step - gen_loss: 6.3685 - disc_loss: 0.2112 - rmse: 0.4111 - val_loss: 0.3017
Epoch 182/2000
1/1 [========

1/1 [==============================] - 0s 135ms/step - gen_loss: 7.9098 - disc_loss: 0.1984 - rmse: 0.2935 - val_loss: 0.3679
Epoch 234/2000
1/1 [==============================] - 0s 145ms/step - gen_loss: 7.7207 - disc_loss: 0.1982 - rmse: 0.3117 - val_loss: 0.3540
Epoch 235/2000
1/1 [==============================] - 0s 132ms/step - gen_loss: 8.1112 - disc_loss: 0.2022 - rmse: 0.2949 - val_loss: 0.2968
Epoch 236/2000
1/1 [==============================] - 0s 148ms/step - gen_loss: 5.1112 - disc_loss: 0.1999 - rmse: 0.2827 - val_loss: 0.2718
Epoch 237/2000
1/1 [==============================] - 0s 136ms/step - gen_loss: 7.0048 - disc_loss: 0.2017 - rmse: 0.3096 - val_loss: 0.4895
Epoch 238/2000
1/1 [==============================] - 0s 373ms/step - gen_loss: 7.0251 - disc_loss: 0.2059 - rmse: 0.3038 - val_loss: 0.4154
Epoch 239/2000
1/1 [==============================] - 0s 143ms/step - gen_loss: 8.0283 - disc_loss: 0.1989 - rmse: 0.3257 - val_loss: 0.2937
Epoch 240/2000
1/1 [========

...

1/1 [==============================] - 0s 162ms/step - gen_loss: 2.5524 - disc_loss: 0.1384 - rmse: 0.2673 - val_loss: 0.2751
Epoch 930/2000
1/1 [==============================] - 0s 143ms/step - gen_loss: 2.6133 - disc_loss: 0.1510 - rmse: 0.3922 - val_loss: 0.2830
Epoch 931/2000
1/1 [==============================] - 0s 97ms/step - gen_loss: 2.5040 - disc_loss: 0.1388 - rmse: 0.3783 - val_loss: 0.2757
Epoch 932/2000
1/1 [==============================] - 0s 134ms/step - gen_loss: 2.6130 - disc_loss: 0.1425 - rmse: 0.3937 - val_loss: 0.2864
Epoch 933/2000
1/1 [==============================] - 0s 118ms/step - gen_loss: 2.4452 - disc_loss: 0.1419 - rmse: 0.3890 - val_loss: 0.3174
Epoch 934/2000
1/1 [==============================] - 0s 357ms/step - gen_loss: 2.6517 - disc_loss: 0.1463 - rmse: 0.5236 - val_loss: 0.3106
Epoch 935/2000
1/1 [==============================] - 0s 128ms/step - gen_loss: 2.9215 - disc_loss: 0.1407 - rmse: 0.2653 - val_loss: 0.3969
Epoch 936/2000
1/1 [=========

1/1 [==============================] - 0s 129ms/step - gen_loss: 2.5796 - disc_loss: 0.1370 - rmse: 0.2447 - val_loss: 0.2814
Epoch 988/2000
1/1 [==============================] - 0s 143ms/step - gen_loss: 2.4932 - disc_loss: 0.1327 - rmse: 0.3062 - val_loss: 0.2709
Epoch 989/2000
1/1 [==============================] - 0s 135ms/step - gen_loss: 2.8238 - disc_loss: 0.1337 - rmse: 0.4305 - val_loss: 0.2565
Epoch 990/2000
1/1 [==============================] - 0s 138ms/step - gen_loss: 2.2139 - disc_loss: 0.1319 - rmse: 0.2411 - val_loss: 0.2717
Epoch 991/2000
1/1 [==============================] - 0s 122ms/step - gen_loss: 2.4050 - disc_loss: 0.1353 - rmse: 0.2598 - val_loss: 0.2658
Epoch 992/2000
1/1 [==============================] - 0s 112ms/step - gen_loss: 2.8148 - disc_loss: 0.1362 - rmse: 0.3732 - val_loss: 0.2535
Epoch 993/2000
1/1 [==============================] - 0s 127ms/step - gen_loss: 2.6398 - disc_loss: 0.1409 - rmse: 0.3878 - val_loss: 0.2723
Epoch 994/2000
1/1 [========

Epoch 1045/2000
1/1 [==============================] - 0s 143ms/step - gen_loss: 2.6125 - disc_loss: 0.1327 - rmse: 0.4592 - val_loss: 0.2451
Epoch 1046/2000
1/1 [==============================] - 0s 92ms/step - gen_loss: 2.5785 - disc_loss: 0.1336 - rmse: 0.5186 - val_loss: 0.2344
Epoch 1047/2000
1/1 [==============================] - 0s 130ms/step - gen_loss: 2.2718 - disc_loss: 0.1359 - rmse: 0.3726 - val_loss: 0.2578
Epoch 1048/2000
1/1 [==============================] - 0s 140ms/step - gen_loss: 2.3378 - disc_loss: 0.1391 - rmse: 0.3135 - val_loss: 0.2517
Epoch 1049/2000
1/1 [==============================] - 0s 109ms/step - gen_loss: 2.1707 - disc_loss: 0.1310 - rmse: 0.2635 - val_loss: 0.2561
Epoch 1050/2000
1/1 [==============================] - 0s 155ms/step - gen_loss: 2.4820 - disc_loss: 0.1330 - rmse: 0.4719 - val_loss: 0.2690
Epoch 1051/2000
1/1 [==============================] - 0s 111ms/step - gen_loss: 2.3565 - disc_loss: 0.1335 - rmse: 0.3863 - val_loss: 0.2363
Epoch 1

1/1 [==============================] - 0s 112ms/step - gen_loss: 2.3439 - disc_loss: 0.1315 - rmse: 0.3740 - val_loss: 0.2631
Epoch 1103/2000
1/1 [==============================] - 0s 124ms/step - gen_loss: 2.0569 - disc_loss: 0.1329 - rmse: 0.4637 - val_loss: 0.2718
Epoch 1104/2000
1/1 [==============================] - 0s 386ms/step - gen_loss: 2.3158 - disc_loss: 0.1341 - rmse: 0.4350 - val_loss: 0.2685
Epoch 1105/2000
1/1 [==============================] - 0s 93ms/step - gen_loss: 2.3108 - disc_loss: 0.1406 - rmse: 0.3644 - val_loss: 0.3176
Epoch 1106/2000
1/1 [==============================] - 0s 127ms/step - gen_loss: 2.2795 - disc_loss: 0.1404 - rmse: 0.4041 - val_loss: 0.3029
Epoch 1107/2000
1/1 [==============================] - 0s 134ms/step - gen_loss: 2.3367 - disc_loss: 0.1401 - rmse: 0.5582 - val_loss: 0.2684
Epoch 1108/2000
1/1 [==============================] - 0s 146ms/step - gen_loss: 2.1028 - disc_loss: 0.1356 - rmse: 0.2569 - val_loss: 0.2745
Epoch 1109/2000
1/1 [==

1/1 [==============================] - 0s 122ms/step - gen_loss: 2.2643 - disc_loss: 0.1376 - rmse: 0.2580 - val_loss: 0.2542
Epoch 1160/2000
1/1 [==============================] - 0s 124ms/step - gen_loss: 2.1353 - disc_loss: 0.1357 - rmse: 0.2518 - val_loss: 0.2593
Epoch 1161/2000
1/1 [==============================] - 0s 157ms/step - gen_loss: 2.4638 - disc_loss: 0.1345 - rmse: 0.2628 - val_loss: 0.2583
Epoch 1162/2000
1/1 [==============================] - 0s 131ms/step - gen_loss: 2.3754 - disc_loss: 0.1351 - rmse: 0.2559 - val_loss: 0.2479
Epoch 1163/2000
1/1 [==============================] - 0s 140ms/step - gen_loss: 2.1374 - disc_loss: 0.1342 - rmse: 0.2539 - val_loss: 0.2512
Epoch 1164/2000
1/1 [==============================] - 0s 107ms/step - gen_loss: 2.0263 - disc_loss: 0.1357 - rmse: 0.2452 - val_loss: 0.2455
Epoch 1165/2000
1/1 [==============================] - 0s 141ms/step - gen_loss: 2.1382 - disc_loss: 0.1355 - rmse: 0.2635 - val_loss: 0.2480
Epoch 1166/2000
1/1 [=

**학습 loss history 출력**

In [38]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax.plot(history.history['gen_loss'], label='gen_loss')
ax.plot(history.history['disc_loss'], label='disc_loss')
ax2.plot(history.history['rmse'], label='rmse', color='green')
ax2.plot(history.history['val_loss'], label='val_loss', color='red')
#plt.legend(history.history.keys(), loc='upper right')
#ax.legend(loc='upper center')
ax.legend(loc='upper center')
ax2.legend(loc='upper right')
ax.set_xlabel("epochs")
ax.set_ylabel("loss")
ax2.set_ylabel("rmse")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

성능 측정

In [39]:
gain.evaluate(wide_window.test.repeat(), steps=100)

100/100 [==============================] - 4s 38ms/step - loss: 0.2609


0.2608625590801239

샘플 prediction 출력

In [40]:
wide_window.plot(gain, plot_col='총유기탄소')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [41]:
norm_df = pd.concat(df,axis=0)

In [42]:
data = norm_df.to_numpy()

total_n = wide_window.dg.data.shape[0]
print(total_n)
unit_shape = wide_window.dg.shape[1:]
print(unit_shape)
#dim = np.prod(wide_window.dg.shape[1:]).astype(int)
dim = wide_window.dg.shape[1]
print(dim)
n = (total_n//dim)*dim

x = data[0:n].copy()
y_true = data[0:n].copy()

#x = interpolate(x, max_gap=3)

print('x.shape =', x.shape)
x_reshape = x.reshape((-1,)+unit_shape)
print('x_reshape.shape =', x_reshape.shape)
isnan = np.isnan(x_reshape)
print(isnan.sum())
print('y_true.shape=', y_true.shape)
isnan = np.isnan(y_true)
print(isnan.sum())

x_remain = data[-wide_window.dg.shape[1]:].copy()
x_remain_reshape = x_remain.reshape((-1,)+unit_shape)
x_remain_reshape.shape

105192
(120, 14)
120
x.shape = (105120, 14)
x_reshape.shape = (876, 120, 14)
235412
y_true.shape= (105120, 14)
235412


(1, 120, 14)

In [43]:
# zero loss is normal because there is no ground truth in the real dataset
gain.evaluate(x_reshape, y_true.reshape((-1,)+unit_shape))

28/28 [==============================] - 0s 3ms/step - loss: 0.0000e+00


0.0

In [44]:
y_pred = gain.predict(x_reshape)
y_remain_pred = gain.predict(x_remain_reshape)

In [45]:
print(total_n)
print(n)
print(total_n-n)

105192
105120
72


In [46]:
y_pred = y_pred.reshape(y_true.shape)
y_remain_pred = y_remain_pred.reshape(x_remain.shape)
print(y_pred.shape, y_remain_pred.shape)
y_pred = np.append(y_pred, y_remain_pred[-(total_n-n):], axis=0)
print(y_pred.shape)

(105120, 14) (120, 14)
(105192, 14)


In [47]:
y_pred[~np.isnan(data)] = np.nan

In [48]:
n = 8
plt.figure(figsize=(9,20))
for i in range(n):
    #plt.subplot('%d1%d'%(n,i))
    plt.subplot(811+i)
    plt.plot(x[:, i])
    plt.plot(y_pred[:, i])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [49]:
total_n = wide_window.dg.data.shape[0]
print(total_n)
unit_shape = wide_window.dg.shape[1:]
print('unit_shape=', unit_shape)
time_seq = unit_shape[0]
print(time_seq)
n = (total_n//time_seq)*time_seq
print('n=', n)

gans = []
oris = []
for i in range(len(df)):
    x = df[i].to_numpy()
    total_n = x.shape[0]
    n = (total_n//time_seq)*time_seq
    x = x[0:n]
    x_block = x.reshape((-1,)+unit_shape)
    y = gain.predict(x_block)
    y_gan = y.reshape(x.shape)
    
    # cut off sin, cos data
    if (i > 0):
        x = x[:, :-4]
        y_gan = y_gan[:, :-4]
    gans.append(y_gan)
    oris.append(x)
print(x.shape)
print(y_gan.shape)

105192
unit_shape= (120, 14)
120
n= 105120
(35040, 10)
(35040, 10)


## 예측 

In [50]:
total_no = oris[0].shape[0]
train_no = int(total_no*0.7)

train_slice = slice(0, train_no)
val_slice = slice(train_no, None)
test_slice = slice(0, None)

train_df = pd.DataFrame(gans[0][train_slice])
val_df = pd.DataFrame(gans[0][val_slice])
test_df = pd.DataFrame(gans[0][test_slice])

train_ori_df = pd.DataFrame(oris[0][train_slice])
val_ori_df = pd.DataFrame(oris[0][val_slice])
test_ori_df = pd.DataFrame(oris[0][test_slice])

num_features = train_df.shape[1]
out_num_features = num_features

In [51]:
ori = np.concatenate(oris, axis=1)
gan = np.concatenate(gans, axis=1)
print(oris[0].shape, gans[0].shape)
print(oris[1].shape, gans[1].shape)
print(ori.shape, gan.shape)

total_no = ori.shape[0]
train_no = int(total_no*0.7)

train_slice = slice(0, train_no)
val_slice = slice(train_no, None)
test_slice = slice(0, None)

train_df = pd.DataFrame(gan[train_slice])
val_df = pd.DataFrame(gan[val_slice])
test_df = pd.DataFrame(gan[test_slice])

train_ori_df = pd.DataFrame(ori[train_slice])
val_ori_df = pd.DataFrame(ori[val_slice])
test_ori_df = pd.DataFrame(ori[test_slice])

num_features = train_df.shape[1]
#out_num_features = oris[0].shape[1]-4
out_features = [6]
out_num_features = len(out_features)
out_num_features


(35040, 14) (35040, 14)
(35040, 10) (35040, 10)
(35040, 34) (35040, 34)


1

In [52]:
class WaterDataGenerator(keras.utils.Sequence):
    'Generates data for water'
    def __init__(self,
                 imputed_data,
                 ori_data = None,
                 batch_size=32,
                 input_width=24*7,
                 label_width=24*3,
                 shift=24*3,
                 skip_time = None,
                 shuffle = True,
                 out_features = None,
                 out_num_features = None,
                ):
        'Initialization'
        self.window_size = input_width+shift
        self.total_no = imputed_data.shape[0]
        self.data = imputed_data
        self.input_width = input_width
        self.label_width = label_width
        self.batch_size = batch_size
        self.input_shape = (batch_size, input_width, self.data.shape[1])
        self.out_num_features = out_num_features
        if out_features:
            self.out_features = out_features
        else:
            self.out_features = [i for i in range(out_num_features)]
        self.label_shape = (batch_size, label_width, self.out_num_features)
        if (skip_time):
            # TO-DO
            self.no = self.total_no - self.window_size
            self.data_idx = np.arange(0, self.no)
        else:
            self.no = self.total_no - self.window_size
            self.data_idx = np.arange(0, self.no)
            
        if shuffle:
            self.batch_idx = np.random.permutation(self.no)
        else:
            self.batch_idx = np.arange(0, self.no)
        self.batch_id = 0
        
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        #return int(128/self.batch_size)
        #return 2
        return 1

    def __getitem__(self, index):
        'Generate one batch of data'
        #print('index =', index)
        #print('self.no =', self.no)
        #print('self.total_no =', self.total_no)
        #print('self.batch_id =', self.batch_id)
        # Sample batch
        label_width = self.label_width
        batch_idx = self.batch_idx
        
        x = np.empty((0, self.input_width, self.data.shape[1]))
        
        #y = np.empty((0, self.input_width, self.data.shape[1]))
        y = np.empty((0, self.label_width, self.out_num_features))
       
        #print(x.shape)
        #print(y.shape)
        
        
        for cnt in range(0, self.batch_size):
            i = self.batch_id
            self.batch_id += 1
            idx1 = self.data_idx[batch_idx[i]]
            idx2 = idx1 + self.input_width
            
            X = self.data[idx1:idx2].to_numpy()
            
            
            idx1 = self.data_idx[batch_idx[i]] + self.window_size - label_width
            idx2 = idx1 + label_width
            
            #Y = self.data[idx1:idx2,:,:out_num_features]
            Y = self.data.iloc[idx1:idx2, self.out_features].to_numpy()
            #Y = self.data[idx1:idx2]
            #print('Y.shape = ', Y.shape)
            #Y = Y.iloc[:,:out_num_features]
            
            self.batch_id %= self.no
            #print("x.shape=", x.shape)
            #print('X.shape=', X.shape)
            #print(type(x), type(X))
            x = np.append(x, [X], axis = 0)
            
            y = np.append(y, [Y], axis = 0)
            
        return x, y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        return

**WaterDataGenerator 테스트**

In [53]:
OUT_STEPS = 24*3
wdg = WaterDataGenerator(train_df,
                         batch_size=128,
                         input_width = 24*7,
                         label_width = OUT_STEPS,
                         shift = OUT_STEPS,
                         out_num_features = out_num_features
                        )

In [54]:
train_df

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
0,-1.251486,0.282415,1.457973,0.150479,-0.192485,-0.558340,1.960215,-0.302267,-0.312016,0.337403,...,1.414207,-1.999586e-12,1.414158,-0.003771,0.033977,0.903974,-1.265062,1.067547,-0.016769,-0.838161
1,-1.278638,0.282415,1.457973,0.150479,-0.204586,-0.558340,2.191326,-0.688669,-0.343491,0.275494,...,1.366019,3.660237e-01,1.414161,-0.002757,0.104390,0.903974,-1.292214,1.067547,-0.016769,-0.838161
2,-1.265062,0.282415,1.457973,0.150479,-0.204586,-0.558340,2.329489,-0.688669,-0.343491,0.337403,...,1.224739,7.071034e-01,1.414162,-0.001743,0.000280,0.903974,-1.332943,0.971013,0.056968,-0.838161
3,-1.265062,0.282415,1.457973,0.150479,-0.192485,-0.558340,2.171229,-0.817470,-0.343491,2.937603,...,0.999995,9.999952e-01,1.414163,-0.000730,0.014116,0.696788,-1.373671,0.971013,0.056968,-0.838161
4,-1.265062,0.282415,1.457973,0.150479,-0.192485,-0.558340,2.098379,-0.688669,-0.343491,-1.519882,...,0.707103,1.224739e+00,1.414163,0.000284,-0.034936,0.696788,-1.400824,0.971013,-0.016769,-0.838161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24523,0.391230,-0.960704,0.646865,-0.573522,0.001122,0.560945,1.126209,0.084135,-0.238574,-0.095963,...,0.366024,-1.366019e+00,0.412795,-1.352624,0.011209,-0.546331,0.282620,-0.380455,-1.638984,0.001303
24524,0.377654,-1.167890,0.573128,-0.670055,-0.010978,0.560945,1.204083,-0.044666,-0.280541,-0.343601,...,0.707103,-1.224739e+00,0.413764,-1.352328,-0.089212,-0.753517,0.241892,-0.525255,-1.638984,0.001303
24525,0.377654,-1.167890,0.573128,-0.670055,-0.010978,0.560945,1.214131,0.084135,-0.280541,-0.343601,...,0.999995,-9.999952e-01,0.414733,-1.352031,-0.114074,-0.960704,0.174011,-0.621789,-1.565247,0.001303
24526,0.377654,-1.167890,0.573128,-0.670055,-0.010978,0.560945,1.221668,0.084135,-0.280541,-0.095963,...,1.224739,-7.071034e-01,0.415702,-1.351733,-0.106484,-0.960704,0.119706,-0.766589,-1.565247,0.001303


In [55]:
it = iter(wdg)

In [56]:
x,y = next(it)
x.shape, y.shape

((128, 168, 34), (128, 72, 1))

In [57]:
def make_dataset_water(self, data):
  dg = WaterDataGenerator(
      data,
      batch_size = 128,
      input_width = self.input_width,
      label_width = self.label_width,
      shift = self.label_width,
      out_features = out_features,
      out_num_features = out_num_features,
  )
  #self.dg = dg
  ds = tf.data.Dataset.from_generator(
      lambda: dg,
      output_types=(tf.float32, tf.float32),
      output_shapes=(
        dg.input_shape,
        dg.label_shape
        #[batch_size, train_generator.dim],
        #[batch_size, train_generator.dim],
      )
  )
  return ds

WindowGenerator.make_dataset = make_dataset_water

In [58]:
def plot2(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0):
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  plot_out_col_index = self.column_indices[plot_out_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]', fontproperties=fprop)
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
      label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
    else:
      label_col_index = plot_col_index
      label_out_col_index = plot_out_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_out_col_index],
                label='Labels', c='#2ca02c')
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_out_col_index],
                  marker=None, label='Predictions',
                  c='#ff7f0e')

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot2 = plot2

In [59]:
multi_window = WindowGenerator(input_width=24*7,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS,
                               train_df=train_df,
                               val_df=val_df,
                               test_df=test_df,
                               )
multi_window

Total window size: 240
Input indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167]
Label indices: [168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185
 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203
 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221
 222 223 224 225 226 227 228 229 230 231 232 

In [60]:
MAX_EPOCHS = 400

def compile_and_fit(model, window, patience=1000):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])
#   model.compile(loss=tf.losses.MeanSquaredError(),
#                 optimizer=tf.optimizers.Adam(),
#                 metrics=[tf.metrics.MeanSquaredError()])
  #model.compile(loss=GAIN.RMSE_loss)

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

In [61]:
def plot_history(history):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(history.history['loss'], label='loss')
    ax.plot(history.history['mean_absolute_error'], label='mae')
#     ax.plot(history.history['mean_squared_error'], label='mse')
    ax.plot(history.history['val_loss'], label='val_loss')
    ax.plot(history.history['val_mean_absolute_error'], label='val_mae')
#     ax.plot(history.history['val_mean_squared_error'], label='val_mse')
    #plt.legend(history.history.keys(), loc='upper right')
    #ax.legend(loc='upper center')
    ax.legend()
    ax.set_xlabel("epochs")
    ax.set_ylabel("loss")
    plt.show()

**Baseline model**

In [62]:
class MultiStepLastBaseline(tf.keras.Model):
  def call(self, inputs):
    #print(inputs[:, -1:, 0:1])
    #return tf.tile(inputs[:, -1:, :out_num_features], [1, OUT_STEPS, 1])
    return tf.tile(inputs[:, -1:, (out_features[0]):(out_features[0]+1)], [1, OUT_STEPS, 1])
    #return tf.tile(inputs[:, -1:, out_features[0]:(out_features[1]+1)], [1, OUT_STEPS, 1])

last_baseline = MultiStepLastBaseline()
last_baseline.compile(loss=tf.losses.MeanSquaredError(),
                      metrics=[tf.metrics.MeanAbsoluteError()])

multi_val_performance = {}
multi_performance = {}

multi_val_performance['BaseLine'] = last_baseline.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['BaseLine'] = last_baseline.evaluate(multi_window.test.repeat(-1), verbose=0, steps=100)
print('val performance =', multi_val_performance['BaseLine'])
print('test performance = ', multi_performance['BaseLine'])


100/100 [==============================] - 8s 84ms/step - loss: 0.0623 - mean_absolute_error: 0.1469
val performance = [0.06072401627898216, 0.1465124487876892]
test performance =  [0.15448619425296783, 0.21511602401733398]


In [63]:
multi_window.plot2(last_baseline, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Linear**

In [64]:
import IPython
multi_linear_model = tf.keras.Sequential([
    # Take the last time-step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.Lambda(lambda x: x[:, -1:, :]),
    # Shape => [batch, 1, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

history = compile_and_fit(multi_linear_model, multi_window)

#IPython.display.clear_output()
multi_val_performance['Linear'] = multi_linear_model.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['Linear'] = multi_linear_model.evaluate(multi_window.test.repeat(-1), verbose=0, steps=100)
#multi_window.plot(multi_linear_model, plot_col=0)
print('val performance =', multi_val_performance['Linear'])
print('test performance = ', multi_performance['Linear'])

Epoch 1/400
1/1 [==============================] - 0s 418ms/step - loss: 1.1720 - mean_absolute_error: 0.8376 - val_loss: 0.4072 - val_mean_absolute_error: 0.5494
Epoch 2/400
1/1 [==============================] - 0s 267ms/step - loss: 1.4077 - mean_absolute_error: 0.8385 - val_loss: 0.4432 - val_mean_absolute_error: 0.5791
Epoch 3/400
1/1 [==============================] - 0s 226ms/step - loss: 1.1968 - mean_absolute_error: 0.7672 - val_loss: 0.3971 - val_mean_absolute_error: 0.5382
Epoch 4/400
1/1 [==============================] - 0s 239ms/step - loss: 1.5162 - mean_absolute_error: 0.8509 - val_loss: 0.3566 - val_mean_absolute_error: 0.5147
Epoch 5/400
1/1 [==============================] - 0s 275ms/step - loss: 1.1848 - mean_absolute_error: 0.7838 - val_loss: 0.3487 - val_mean_absolute_error: 0.5218
Epoch 6/400
1/1 [==============================] - 0s 261ms/step - loss: 1.4619 - mean_absolute_error: 0.8472 - val_loss: 0.3207 - val_mean_absolute_error: 0.4958
Epoch 7/400
1/1 [=====

1/1 [==============================] - 0s 258ms/step - loss: 1.1037 - mean_absolute_error: 0.7344 - val_loss: 0.2586 - val_mean_absolute_error: 0.4151
Epoch 52/400
1/1 [==============================] - 0s 208ms/step - loss: 0.9702 - mean_absolute_error: 0.7090 - val_loss: 0.3462 - val_mean_absolute_error: 0.4822
Epoch 53/400
1/1 [==============================] - 0s 264ms/step - loss: 0.8091 - mean_absolute_error: 0.6276 - val_loss: 0.3186 - val_mean_absolute_error: 0.4437
Epoch 54/400
1/1 [==============================] - 0s 258ms/step - loss: 1.0080 - mean_absolute_error: 0.6846 - val_loss: 0.2640 - val_mean_absolute_error: 0.4186
Epoch 55/400
1/1 [==============================] - 0s 224ms/step - loss: 0.9577 - mean_absolute_error: 0.6482 - val_loss: 0.3452 - val_mean_absolute_error: 0.4811
Epoch 56/400
1/1 [==============================] - 0s 241ms/step - loss: 0.6698 - mean_absolute_error: 0.5875 - val_loss: 0.2559 - val_mean_absolute_error: 0.4075
Epoch 57/400
1/1 [===========

1/1 [==============================] - 0s 247ms/step - loss: 0.7557 - mean_absolute_error: 0.5634 - val_loss: 0.2772 - val_mean_absolute_error: 0.4215
Epoch 102/400
1/1 [==============================] - 0s 221ms/step - loss: 0.6364 - mean_absolute_error: 0.5603 - val_loss: 0.2879 - val_mean_absolute_error: 0.4194
Epoch 103/400
1/1 [==============================] - 0s 244ms/step - loss: 0.7741 - mean_absolute_error: 0.6401 - val_loss: 0.2903 - val_mean_absolute_error: 0.4372
Epoch 104/400
1/1 [==============================] - 0s 259ms/step - loss: 0.8378 - mean_absolute_error: 0.6621 - val_loss: 0.2809 - val_mean_absolute_error: 0.4408
Epoch 105/400
1/1 [==============================] - 0s 255ms/step - loss: 0.6071 - mean_absolute_error: 0.5867 - val_loss: 0.3334 - val_mean_absolute_error: 0.4551
Epoch 106/400
1/1 [==============================] - 0s 234ms/step - loss: 0.9122 - mean_absolute_error: 0.6647 - val_loss: 0.2570 - val_mean_absolute_error: 0.3997
Epoch 107/400
1/1 [=====

Epoch 151/400
1/1 [==============================] - 0s 234ms/step - loss: 0.5299 - mean_absolute_error: 0.5387 - val_loss: 0.2001 - val_mean_absolute_error: 0.3488
Epoch 152/400
1/1 [==============================] - 0s 227ms/step - loss: 0.5556 - mean_absolute_error: 0.5352 - val_loss: 0.2306 - val_mean_absolute_error: 0.3604
Epoch 153/400
1/1 [==============================] - 0s 251ms/step - loss: 0.5803 - mean_absolute_error: 0.5250 - val_loss: 0.2111 - val_mean_absolute_error: 0.3648
Epoch 154/400
1/1 [==============================] - 0s 282ms/step - loss: 0.6123 - mean_absolute_error: 0.5512 - val_loss: 0.2011 - val_mean_absolute_error: 0.3605
Epoch 155/400
1/1 [==============================] - 0s 239ms/step - loss: 0.8936 - mean_absolute_error: 0.6007 - val_loss: 0.1877 - val_mean_absolute_error: 0.3356
Epoch 156/400
1/1 [==============================] - 0s 239ms/step - loss: 0.6537 - mean_absolute_error: 0.5600 - val_loss: 0.1857 - val_mean_absolute_error: 0.3338
Epoch 157/

Epoch 201/400
1/1 [==============================] - 0s 233ms/step - loss: 0.5603 - mean_absolute_error: 0.5301 - val_loss: 0.2123 - val_mean_absolute_error: 0.3640
Epoch 202/400
1/1 [==============================] - 0s 241ms/step - loss: 0.5751 - mean_absolute_error: 0.5274 - val_loss: 0.1801 - val_mean_absolute_error: 0.3420
Epoch 203/400
1/1 [==============================] - 0s 266ms/step - loss: 0.4875 - mean_absolute_error: 0.4954 - val_loss: 0.1934 - val_mean_absolute_error: 0.3435
Epoch 204/400
1/1 [==============================] - 0s 215ms/step - loss: 0.3832 - mean_absolute_error: 0.4361 - val_loss: 0.1733 - val_mean_absolute_error: 0.3252
Epoch 205/400
1/1 [==============================] - 0s 248ms/step - loss: 0.3284 - mean_absolute_error: 0.4185 - val_loss: 0.1697 - val_mean_absolute_error: 0.3242
Epoch 206/400
1/1 [==============================] - 0s 232ms/step - loss: 0.6203 - mean_absolute_error: 0.5379 - val_loss: 0.1973 - val_mean_absolute_error: 0.3457
Epoch 207/

Epoch 251/400
1/1 [==============================] - 0s 223ms/step - loss: 0.5487 - mean_absolute_error: 0.4699 - val_loss: 0.1560 - val_mean_absolute_error: 0.3224
Epoch 252/400
1/1 [==============================] - 0s 206ms/step - loss: 0.4040 - mean_absolute_error: 0.4580 - val_loss: 0.1724 - val_mean_absolute_error: 0.3326
Epoch 253/400
1/1 [==============================] - 0s 248ms/step - loss: 0.4712 - mean_absolute_error: 0.4708 - val_loss: 0.1466 - val_mean_absolute_error: 0.2991
Epoch 254/400
1/1 [==============================] - 0s 231ms/step - loss: 0.4419 - mean_absolute_error: 0.4591 - val_loss: 0.1735 - val_mean_absolute_error: 0.3140
Epoch 255/400
1/1 [==============================] - 0s 192ms/step - loss: 0.4473 - mean_absolute_error: 0.4194 - val_loss: 0.1755 - val_mean_absolute_error: 0.3355
Epoch 256/400
1/1 [==============================] - 0s 230ms/step - loss: 0.4798 - mean_absolute_error: 0.4840 - val_loss: 0.1987 - val_mean_absolute_error: 0.3492
Epoch 257/

Epoch 301/400
1/1 [==============================] - 0s 238ms/step - loss: 0.2693 - mean_absolute_error: 0.3715 - val_loss: 0.1516 - val_mean_absolute_error: 0.3073
Epoch 302/400
1/1 [==============================] - 0s 233ms/step - loss: 0.3151 - mean_absolute_error: 0.4138 - val_loss: 0.1801 - val_mean_absolute_error: 0.3024
Epoch 303/400
1/1 [==============================] - 0s 238ms/step - loss: 0.3330 - mean_absolute_error: 0.4151 - val_loss: 0.1418 - val_mean_absolute_error: 0.3053
Epoch 304/400
1/1 [==============================] - 0s 260ms/step - loss: 0.3467 - mean_absolute_error: 0.4191 - val_loss: 0.1361 - val_mean_absolute_error: 0.2949
Epoch 305/400
1/1 [==============================] - 0s 238ms/step - loss: 0.4384 - mean_absolute_error: 0.4349 - val_loss: 0.1712 - val_mean_absolute_error: 0.3142
Epoch 306/400
1/1 [==============================] - 0s 219ms/step - loss: 0.3337 - mean_absolute_error: 0.4104 - val_loss: 0.1301 - val_mean_absolute_error: 0.2746
Epoch 307/

Epoch 351/400
1/1 [==============================] - 0s 241ms/step - loss: 0.4389 - mean_absolute_error: 0.4753 - val_loss: 0.1288 - val_mean_absolute_error: 0.2832
Epoch 352/400
1/1 [==============================] - 0s 221ms/step - loss: 0.6274 - mean_absolute_error: 0.4425 - val_loss: 0.1418 - val_mean_absolute_error: 0.2983
Epoch 353/400
1/1 [==============================] - 0s 252ms/step - loss: 0.3421 - mean_absolute_error: 0.4022 - val_loss: 0.1764 - val_mean_absolute_error: 0.3165
Epoch 354/400
1/1 [==============================] - 0s 227ms/step - loss: 0.5410 - mean_absolute_error: 0.4661 - val_loss: 0.1230 - val_mean_absolute_error: 0.2813
Epoch 355/400
1/1 [==============================] - 0s 296ms/step - loss: 0.4162 - mean_absolute_error: 0.4431 - val_loss: 0.1740 - val_mean_absolute_error: 0.3198
Epoch 356/400
1/1 [==============================] - 0s 264ms/step - loss: 0.3218 - mean_absolute_error: 0.4227 - val_loss: 0.1941 - val_mean_absolute_error: 0.3449
Epoch 357/

100/100 [==============================] - 8s 85ms/step - loss: 0.1506 - mean_absolute_error: 0.3063
val performance = [0.15062035620212555, 0.3063274025917053]
test performance =  [0.29991114139556885, 0.3786599636077881]


In [65]:
multi_performance['Linear'] = multi_linear_model.evaluate(multi_window.test.repeat(-1), verbose=0, steps=100)

In [66]:
plot_history(history)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [67]:
multi_window.plot2(multi_linear_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Elman_RNN**

In [68]:
elman_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.SimpleRNN(128, return_sequences=False),

    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),

    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [69]:
elman_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.SimpleRNN(128, return_sequences=True),
    tf.keras.layers.SimpleRNN(128, return_sequences=True),
    tf.keras.layers.SimpleRNN(128, return_sequences=True),
    tf.keras.layers.SimpleRNN(128, return_sequences=True),
    tf.keras.layers.SimpleRNN(128, return_sequences=True),
    tf.keras.layers.SimpleRNN(128, return_sequences=True),
    tf.keras.layers.SimpleRNN(128, return_sequences=True),
    tf.keras.layers.SimpleRNN(128, return_sequences=True),
    tf.keras.layers.SimpleRNN(128, return_sequences=False),

    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),

    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [70]:
history = compile_and_fit(elman_model, multi_window)
multi_val_performance['ELMAN_RNN'] = elman_model.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['ELMAN_RNN'] = elman_model.evaluate(multi_window.test.repeat(-1), verbose=1, steps=100)
print('val performance =', multi_val_performance['ELMAN_RNN'])
print('test performance = ', multi_performance['ELMAN_RNN'])

Epoch 1/400
1/1 [==============================] - 4s 4s/step - loss: 1.3415 - mean_absolute_error: 0.8576 - val_loss: 0.4167 - val_mean_absolute_error: 0.5594
Epoch 2/400
1/1 [==============================] - 1s 616ms/step - loss: 1.1485 - mean_absolute_error: 0.8326 - val_loss: 0.3755 - val_mean_absolute_error: 0.5271
Epoch 3/400
1/1 [==============================] - 1s 624ms/step - loss: 1.8449 - mean_absolute_error: 0.9249 - val_loss: 0.2551 - val_mean_absolute_error: 0.4175
Epoch 4/400
1/1 [==============================] - 1s 611ms/step - loss: 1.0882 - mean_absolute_error: 0.7417 - val_loss: 0.2879 - val_mean_absolute_error: 0.4480
Epoch 5/400
1/1 [==============================] - 1s 582ms/step - loss: 1.6863 - mean_absolute_error: 0.8659 - val_loss: 0.3047 - val_mean_absolute_error: 0.4403
Epoch 6/400
1/1 [==============================] - 1s 598ms/step - loss: 0.8528 - mean_absolute_error: 0.5981 - val_loss: 0.2100 - val_mean_absolute_error: 0.3454
Epoch 7/400
1/1 [========

1/1 [==============================] - 1s 549ms/step - loss: 0.2354 - mean_absolute_error: 0.3241 - val_loss: 0.1055 - val_mean_absolute_error: 0.2354
Epoch 52/400
1/1 [==============================] - 1s 590ms/step - loss: 0.3655 - mean_absolute_error: 0.3750 - val_loss: 0.0970 - val_mean_absolute_error: 0.2284
Epoch 53/400
1/1 [==============================] - 1s 627ms/step - loss: 0.4504 - mean_absolute_error: 0.3969 - val_loss: 0.1723 - val_mean_absolute_error: 0.3029
Epoch 54/400
1/1 [==============================] - 1s 562ms/step - loss: 0.3129 - mean_absolute_error: 0.3956 - val_loss: 0.2703 - val_mean_absolute_error: 0.4145
Epoch 55/400
1/1 [==============================] - 1s 588ms/step - loss: 0.2494 - mean_absolute_error: 0.3926 - val_loss: 0.2669 - val_mean_absolute_error: 0.4058
Epoch 56/400
1/1 [==============================] - 1s 607ms/step - loss: 0.4827 - mean_absolute_error: 0.4304 - val_loss: 0.1456 - val_mean_absolute_error: 0.2815
Epoch 57/400
1/1 [===========

1/1 [==============================] - 1s 637ms/step - loss: 0.3694 - mean_absolute_error: 0.3919 - val_loss: 0.2836 - val_mean_absolute_error: 0.4028
Epoch 102/400
1/1 [==============================] - 1s 591ms/step - loss: 0.8478 - mean_absolute_error: 0.6117 - val_loss: 0.3900 - val_mean_absolute_error: 0.5021
Epoch 103/400
1/1 [==============================] - 1s 606ms/step - loss: 0.7663 - mean_absolute_error: 0.6308 - val_loss: 0.4665 - val_mean_absolute_error: 0.5596
Epoch 104/400
1/1 [==============================] - 1s 626ms/step - loss: 0.5224 - mean_absolute_error: 0.4971 - val_loss: 0.4839 - val_mean_absolute_error: 0.5484
Epoch 105/400
1/1 [==============================] - 1s 600ms/step - loss: 0.7139 - mean_absolute_error: 0.6346 - val_loss: 0.4134 - val_mean_absolute_error: 0.5033
Epoch 106/400
1/1 [==============================] - 1s 601ms/step - loss: 0.8377 - mean_absolute_error: 0.6184 - val_loss: 0.3871 - val_mean_absolute_error: 0.4944
Epoch 107/400
1/1 [=====

Epoch 151/400
1/1 [==============================] - 1s 598ms/step - loss: 0.4500 - mean_absolute_error: 0.3891 - val_loss: 0.1197 - val_mean_absolute_error: 0.2338
Epoch 152/400
1/1 [==============================] - 1s 583ms/step - loss: 0.2144 - mean_absolute_error: 0.3043 - val_loss: 0.0818 - val_mean_absolute_error: 0.2110
Epoch 153/400
1/1 [==============================] - 1s 631ms/step - loss: 0.2400 - mean_absolute_error: 0.3592 - val_loss: 0.0858 - val_mean_absolute_error: 0.2109
Epoch 154/400
1/1 [==============================] - 1s 590ms/step - loss: 0.2160 - mean_absolute_error: 0.3142 - val_loss: 0.1190 - val_mean_absolute_error: 0.2549
Epoch 155/400
1/1 [==============================] - 1s 594ms/step - loss: 0.2093 - mean_absolute_error: 0.3109 - val_loss: 0.1351 - val_mean_absolute_error: 0.2779
Epoch 156/400
1/1 [==============================] - 1s 622ms/step - loss: 0.3497 - mean_absolute_error: 0.3819 - val_loss: 0.0733 - val_mean_absolute_error: 0.2008
Epoch 157/

1/1 [==============================] - 1s 559ms/step - loss: 0.1356 - mean_absolute_error: 0.2632 - val_loss: 0.0812 - val_mean_absolute_error: 0.2142
Epoch 201/400
1/1 [==============================] - 1s 597ms/step - loss: 0.1851 - mean_absolute_error: 0.3027 - val_loss: 0.1113 - val_mean_absolute_error: 0.2639
Epoch 202/400
1/1 [==============================] - 1s 596ms/step - loss: 0.4298 - mean_absolute_error: 0.3548 - val_loss: 0.0979 - val_mean_absolute_error: 0.2049
Epoch 203/400
1/1 [==============================] - 1s 584ms/step - loss: 0.3064 - mean_absolute_error: 0.3120 - val_loss: 0.0887 - val_mean_absolute_error: 0.2220
Epoch 204/400
1/1 [==============================] - 1s 591ms/step - loss: 0.2624 - mean_absolute_error: 0.2859 - val_loss: 0.1173 - val_mean_absolute_error: 0.2545
Epoch 205/400
1/1 [==============================] - 1s 640ms/step - loss: 0.2333 - mean_absolute_error: 0.3365 - val_loss: 0.1503 - val_mean_absolute_error: 0.2979
Epoch 206/400
1/1 [=====

Epoch 250/400
1/1 [==============================] - 1s 643ms/step - loss: 0.1574 - mean_absolute_error: 0.2941 - val_loss: 0.0717 - val_mean_absolute_error: 0.1943
Epoch 251/400
1/1 [==============================] - 1s 581ms/step - loss: 0.1098 - mean_absolute_error: 0.2321 - val_loss: 0.0619 - val_mean_absolute_error: 0.1872
Epoch 252/400
1/1 [==============================] - 1s 583ms/step - loss: 0.2213 - mean_absolute_error: 0.2479 - val_loss: 0.0930 - val_mean_absolute_error: 0.2203
Epoch 253/400
1/1 [==============================] - 1s 613ms/step - loss: 0.1139 - mean_absolute_error: 0.2294 - val_loss: 0.0845 - val_mean_absolute_error: 0.2206
Epoch 254/400
1/1 [==============================] - 1s 599ms/step - loss: 0.1460 - mean_absolute_error: 0.2820 - val_loss: 0.0764 - val_mean_absolute_error: 0.1952
Epoch 255/400
1/1 [==============================] - 1s 600ms/step - loss: 0.2047 - mean_absolute_error: 0.3000 - val_loss: 0.0644 - val_mean_absolute_error: 0.1884
Epoch 256/

Epoch 300/400
1/1 [==============================] - 1s 589ms/step - loss: 0.2489 - mean_absolute_error: 0.2515 - val_loss: 0.0649 - val_mean_absolute_error: 0.1954
Epoch 301/400
1/1 [==============================] - 1s 606ms/step - loss: 0.2326 - mean_absolute_error: 0.2917 - val_loss: 0.0844 - val_mean_absolute_error: 0.2038
Epoch 302/400
1/1 [==============================] - 1s 572ms/step - loss: 0.2645 - mean_absolute_error: 0.2665 - val_loss: 0.0545 - val_mean_absolute_error: 0.1766
Epoch 303/400
1/1 [==============================] - 1s 612ms/step - loss: 0.3516 - mean_absolute_error: 0.2816 - val_loss: 0.1022 - val_mean_absolute_error: 0.2251
Epoch 304/400
1/1 [==============================] - 1s 579ms/step - loss: 0.1267 - mean_absolute_error: 0.2374 - val_loss: 0.0729 - val_mean_absolute_error: 0.2005
Epoch 305/400
1/1 [==============================] - 1s 589ms/step - loss: 0.1108 - mean_absolute_error: 0.2385 - val_loss: 0.1011 - val_mean_absolute_error: 0.2178
Epoch 306/

Epoch 350/400
1/1 [==============================] - 1s 588ms/step - loss: 0.1104 - mean_absolute_error: 0.2234 - val_loss: 0.0833 - val_mean_absolute_error: 0.2187
Epoch 351/400
1/1 [==============================] - 1s 585ms/step - loss: 0.1255 - mean_absolute_error: 0.2482 - val_loss: 0.0964 - val_mean_absolute_error: 0.2254
Epoch 352/400
1/1 [==============================] - 1s 590ms/step - loss: 0.1286 - mean_absolute_error: 0.2413 - val_loss: 0.0760 - val_mean_absolute_error: 0.1967
Epoch 353/400
1/1 [==============================] - 1s 578ms/step - loss: 0.1680 - mean_absolute_error: 0.2607 - val_loss: 0.0638 - val_mean_absolute_error: 0.1966
Epoch 354/400
1/1 [==============================] - 1s 596ms/step - loss: 0.1207 - mean_absolute_error: 0.2488 - val_loss: 0.0642 - val_mean_absolute_error: 0.1949
Epoch 355/400
1/1 [==============================] - 1s 576ms/step - loss: 0.2696 - mean_absolute_error: 0.2935 - val_loss: 0.1011 - val_mean_absolute_error: 0.2176
Epoch 356/

Epoch 400/400
100/100 [==============================] - 15s 149ms/step - loss: 0.1351 - mean_absolute_error: 0.2355
val performance = [0.07991129904985428, 0.206924170255661]
test performance =  [0.13506361842155457, 0.23548410832881927]


In [71]:
plot_history(history)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [72]:
multi_window.plot2(elman_model, plot_col=5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [73]:
multi_window.plot2(elman_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**GRU**

In [74]:
gru_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.GRU(128, return_sequences=False),
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [75]:
MAX_EPOCHS = 450

In [76]:
history = compile_and_fit(gru_model, multi_window)
multi_val_performance['GRU'] = gru_model.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['GRU'] = gru_model.evaluate(multi_window.test.repeat(-1), verbose=1, steps=100)
print('val performance =', multi_val_performance['GRU'])
print('test performance = ', multi_performance['GRU'])

Epoch 1/450
1/1 [==============================] - 5s 5s/step - loss: 1.3241 - mean_absolute_error: 0.8436 - val_loss: 0.4404 - val_mean_absolute_error: 0.5891
Epoch 2/450
1/1 [==============================] - 1s 882ms/step - loss: 1.3050 - mean_absolute_error: 0.7942 - val_loss: 0.4411 - val_mean_absolute_error: 0.5707
Epoch 3/450
1/1 [==============================] - 1s 888ms/step - loss: 1.0193 - mean_absolute_error: 0.7625 - val_loss: 0.3664 - val_mean_absolute_error: 0.5210
Epoch 4/450
1/1 [==============================] - 1s 875ms/step - loss: 1.3676 - mean_absolute_error: 0.8518 - val_loss: 0.3625 - val_mean_absolute_error: 0.5209
Epoch 5/450
1/1 [==============================] - 1s 909ms/step - loss: 1.1470 - mean_absolute_error: 0.7139 - val_loss: 0.3268 - val_mean_absolute_error: 0.4878
Epoch 6/450
1/1 [==============================] - 1s 866ms/step - loss: 1.4930 - mean_absolute_error: 0.8486 - val_loss: 0.3035 - val_mean_absolute_error: 0.4395
Epoch 7/450
1/1 [========

1/1 [==============================] - 1s 930ms/step - loss: 0.1742 - mean_absolute_error: 0.2692 - val_loss: 0.0499 - val_mean_absolute_error: 0.1547
Epoch 52/450
1/1 [==============================] - 1s 937ms/step - loss: 0.2292 - mean_absolute_error: 0.2863 - val_loss: 0.0869 - val_mean_absolute_error: 0.2189
Epoch 53/450
1/1 [==============================] - 1s 920ms/step - loss: 0.2029 - mean_absolute_error: 0.2905 - val_loss: 0.0828 - val_mean_absolute_error: 0.2249
Epoch 54/450
1/1 [==============================] - 1s 915ms/step - loss: 0.2106 - mean_absolute_error: 0.2927 - val_loss: 0.1006 - val_mean_absolute_error: 0.2286
Epoch 55/450
1/1 [==============================] - 1s 941ms/step - loss: 0.2683 - mean_absolute_error: 0.3207 - val_loss: 0.0716 - val_mean_absolute_error: 0.1978
Epoch 56/450
1/1 [==============================] - 1s 895ms/step - loss: 0.2493 - mean_absolute_error: 0.3218 - val_loss: 0.0568 - val_mean_absolute_error: 0.1774
Epoch 57/450
1/1 [===========

1/1 [==============================] - 1s 945ms/step - loss: 0.1543 - mean_absolute_error: 0.2747 - val_loss: 0.1389 - val_mean_absolute_error: 0.2914
Epoch 102/450
1/1 [==============================] - 1s 899ms/step - loss: 0.1534 - mean_absolute_error: 0.2822 - val_loss: 0.0981 - val_mean_absolute_error: 0.2310
Epoch 103/450
1/1 [==============================] - 1s 860ms/step - loss: 0.1418 - mean_absolute_error: 0.2492 - val_loss: 0.0907 - val_mean_absolute_error: 0.2148
Epoch 104/450
1/1 [==============================] - 1s 880ms/step - loss: 0.1397 - mean_absolute_error: 0.2612 - val_loss: 0.0724 - val_mean_absolute_error: 0.1960
Epoch 105/450
1/1 [==============================] - 1s 892ms/step - loss: 0.3606 - mean_absolute_error: 0.2766 - val_loss: 0.0427 - val_mean_absolute_error: 0.1603
Epoch 106/450
1/1 [==============================] - 1s 905ms/step - loss: 0.2038 - mean_absolute_error: 0.2848 - val_loss: 0.0674 - val_mean_absolute_error: 0.1722
Epoch 107/450
1/1 [=====

Epoch 151/450
1/1 [==============================] - 1s 897ms/step - loss: 0.1449 - mean_absolute_error: 0.2593 - val_loss: 0.0771 - val_mean_absolute_error: 0.2040
Epoch 152/450
1/1 [==============================] - 1s 918ms/step - loss: 0.1332 - mean_absolute_error: 0.2519 - val_loss: 0.0608 - val_mean_absolute_error: 0.1906
Epoch 153/450
1/1 [==============================] - 1s 867ms/step - loss: 0.0903 - mean_absolute_error: 0.2075 - val_loss: 0.0638 - val_mean_absolute_error: 0.1924
Epoch 154/450
1/1 [==============================] - 1s 855ms/step - loss: 0.0869 - mean_absolute_error: 0.2073 - val_loss: 0.0796 - val_mean_absolute_error: 0.2130
Epoch 155/450
1/1 [==============================] - 1s 919ms/step - loss: 0.1617 - mean_absolute_error: 0.2583 - val_loss: 0.0555 - val_mean_absolute_error: 0.1820
Epoch 156/450
1/1 [==============================] - 1s 907ms/step - loss: 0.1234 - mean_absolute_error: 0.2360 - val_loss: 0.0955 - val_mean_absolute_error: 0.2284
Epoch 157/

Epoch 201/450
1/1 [==============================] - 1s 897ms/step - loss: 0.1250 - mean_absolute_error: 0.2490 - val_loss: 0.0814 - val_mean_absolute_error: 0.2259
Epoch 202/450
1/1 [==============================] - 1s 895ms/step - loss: 0.1119 - mean_absolute_error: 0.2330 - val_loss: 0.0986 - val_mean_absolute_error: 0.2324
Epoch 203/450
1/1 [==============================] - 1s 880ms/step - loss: 0.2179 - mean_absolute_error: 0.2315 - val_loss: 0.0889 - val_mean_absolute_error: 0.2241
Epoch 204/450
1/1 [==============================] - 1s 896ms/step - loss: 0.1589 - mean_absolute_error: 0.2224 - val_loss: 0.0754 - val_mean_absolute_error: 0.1972
Epoch 205/450
1/1 [==============================] - 1s 917ms/step - loss: 0.4981 - mean_absolute_error: 0.2992 - val_loss: 0.0881 - val_mean_absolute_error: 0.2282
Epoch 206/450
1/1 [==============================] - 1s 911ms/step - loss: 0.1303 - mean_absolute_error: 0.2502 - val_loss: 0.0811 - val_mean_absolute_error: 0.2181
Epoch 207/

Epoch 251/450
1/1 [==============================] - 1s 878ms/step - loss: 0.1011 - mean_absolute_error: 0.2180 - val_loss: 0.0717 - val_mean_absolute_error: 0.2031
Epoch 252/450
1/1 [==============================] - 1s 888ms/step - loss: 0.0929 - mean_absolute_error: 0.2111 - val_loss: 0.0996 - val_mean_absolute_error: 0.2175
Epoch 253/450
1/1 [==============================] - 1s 919ms/step - loss: 0.1351 - mean_absolute_error: 0.2115 - val_loss: 0.0761 - val_mean_absolute_error: 0.1939
Epoch 254/450
1/1 [==============================] - 1s 921ms/step - loss: 0.1070 - mean_absolute_error: 0.2219 - val_loss: 0.0962 - val_mean_absolute_error: 0.2130
Epoch 255/450
1/1 [==============================] - 1s 862ms/step - loss: 0.1097 - mean_absolute_error: 0.2295 - val_loss: 0.1144 - val_mean_absolute_error: 0.2213
Epoch 256/450
1/1 [==============================] - 1s 910ms/step - loss: 0.2529 - mean_absolute_error: 0.2368 - val_loss: 0.0613 - val_mean_absolute_error: 0.1830
Epoch 257/

Epoch 301/450
1/1 [==============================] - 1s 933ms/step - loss: 0.0902 - mean_absolute_error: 0.2148 - val_loss: 0.1090 - val_mean_absolute_error: 0.2406
Epoch 302/450
1/1 [==============================] - 1s 892ms/step - loss: 0.1226 - mean_absolute_error: 0.2453 - val_loss: 0.0773 - val_mean_absolute_error: 0.2160
Epoch 303/450
1/1 [==============================] - 1s 917ms/step - loss: 0.0936 - mean_absolute_error: 0.2051 - val_loss: 0.0838 - val_mean_absolute_error: 0.2173
Epoch 304/450
1/1 [==============================] - 1s 902ms/step - loss: 0.1319 - mean_absolute_error: 0.2313 - val_loss: 0.0789 - val_mean_absolute_error: 0.2049
Epoch 305/450
1/1 [==============================] - 1s 871ms/step - loss: 0.1218 - mean_absolute_error: 0.2321 - val_loss: 0.0856 - val_mean_absolute_error: 0.2092
Epoch 306/450
1/1 [==============================] - 1s 867ms/step - loss: 0.1393 - mean_absolute_error: 0.2428 - val_loss: 0.0845 - val_mean_absolute_error: 0.2075
Epoch 307/

Epoch 351/450
1/1 [==============================] - 1s 881ms/step - loss: 0.2302 - mean_absolute_error: 0.2565 - val_loss: 0.0936 - val_mean_absolute_error: 0.2189
Epoch 352/450
1/1 [==============================] - 1s 928ms/step - loss: 0.1552 - mean_absolute_error: 0.2378 - val_loss: 0.0690 - val_mean_absolute_error: 0.1942
Epoch 353/450
1/1 [==============================] - 1s 905ms/step - loss: 0.3535 - mean_absolute_error: 0.2646 - val_loss: 0.0850 - val_mean_absolute_error: 0.2171
Epoch 354/450
1/1 [==============================] - 1s 882ms/step - loss: 0.1013 - mean_absolute_error: 0.2260 - val_loss: 0.0659 - val_mean_absolute_error: 0.1979
Epoch 355/450
1/1 [==============================] - 1s 900ms/step - loss: 0.0792 - mean_absolute_error: 0.2039 - val_loss: 0.0774 - val_mean_absolute_error: 0.1959
Epoch 356/450
1/1 [==============================] - 1s 920ms/step - loss: 0.1063 - mean_absolute_error: 0.2254 - val_loss: 0.0808 - val_mean_absolute_error: 0.1870
Epoch 357/

Epoch 401/450
1/1 [==============================] - 1s 889ms/step - loss: 0.1110 - mean_absolute_error: 0.2341 - val_loss: 0.0477 - val_mean_absolute_error: 0.1527
Epoch 402/450
1/1 [==============================] - 1s 912ms/step - loss: 0.2505 - mean_absolute_error: 0.2556 - val_loss: 0.0546 - val_mean_absolute_error: 0.1690
Epoch 403/450
1/1 [==============================] - 1s 933ms/step - loss: 0.0963 - mean_absolute_error: 0.2255 - val_loss: 0.0953 - val_mean_absolute_error: 0.2224
Epoch 404/450
1/1 [==============================] - 1s 836ms/step - loss: 0.2701 - mean_absolute_error: 0.2542 - val_loss: 0.1010 - val_mean_absolute_error: 0.2247
Epoch 405/450
1/1 [==============================] - 1s 893ms/step - loss: 0.1425 - mean_absolute_error: 0.2579 - val_loss: 0.1049 - val_mean_absolute_error: 0.2233
Epoch 406/450
1/1 [==============================] - 1s 919ms/step - loss: 0.1955 - mean_absolute_error: 0.2555 - val_loss: 0.0993 - val_mean_absolute_error: 0.2288
Epoch 407/

100/100 [==============================] - 21s 214ms/step - loss: 0.1126 - mean_absolute_error: 0.2146
val performance = [0.09623899310827255, 0.2325861006975174]
test performance =  [0.11256847530603409, 0.21460266411304474]


In [77]:
plot_history(history)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [78]:
multi_window.plot2(gru_model, plot_col=3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [79]:
multi_window.plot2(gru_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**LSTM**

In [80]:
multi_lstm_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, lstm_units]
    # Adding more `lstm_units` just overfits more quickly.
    #tf.keras.layers.LSTM(32, return_sequences=False),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(128, return_sequences=False),
    # Shape => [batch, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [81]:
history = compile_and_fit(multi_lstm_model, multi_window)

#IPython.display.clear_output()

multi_val_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.val)
multi_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['LSTM'])
print('test performance = ', multi_performance['LSTM'])

Epoch 1/450
1/1 [==============================] - 11s 11s/step - loss: 1.0401 - mean_absolute_error: 0.7670 - val_loss: 0.3646 - val_mean_absolute_error: 0.5229
Epoch 2/450
1/1 [==============================] - 0s 448ms/step - loss: 1.6922 - mean_absolute_error: 0.8708 - val_loss: 0.3566 - val_mean_absolute_error: 0.5059
Epoch 3/450
1/1 [==============================] - 0s 461ms/step - loss: 1.5625 - mean_absolute_error: 0.8545 - val_loss: 0.3961 - val_mean_absolute_error: 0.5419
Epoch 4/450
1/1 [==============================] - 0s 434ms/step - loss: 1.3863 - mean_absolute_error: 0.8686 - val_loss: 0.3219 - val_mean_absolute_error: 0.4639
Epoch 5/450
1/1 [==============================] - 0s 389ms/step - loss: 1.7736 - mean_absolute_error: 0.9460 - val_loss: 0.2959 - val_mean_absolute_error: 0.4591
Epoch 6/450
1/1 [==============================] - 0s 455ms/step - loss: 1.1886 - mean_absolute_error: 0.7920 - val_loss: 0.2505 - val_mean_absolute_error: 0.3973
Epoch 7/450
1/1 [======

1/1 [==============================] - 0s 465ms/step - loss: 0.9524 - mean_absolute_error: 0.6914 - val_loss: 0.3676 - val_mean_absolute_error: 0.5343
Epoch 52/450
1/1 [==============================] - 0s 431ms/step - loss: 0.9678 - mean_absolute_error: 0.7244 - val_loss: 0.4752 - val_mean_absolute_error: 0.6063
Epoch 53/450
1/1 [==============================] - 0s 452ms/step - loss: 1.0071 - mean_absolute_error: 0.6919 - val_loss: 0.3976 - val_mean_absolute_error: 0.5435
Epoch 54/450
1/1 [==============================] - 0s 480ms/step - loss: 0.9650 - mean_absolute_error: 0.7014 - val_loss: 0.4143 - val_mean_absolute_error: 0.5577
Epoch 55/450
1/1 [==============================] - 0s 395ms/step - loss: 0.7813 - mean_absolute_error: 0.6108 - val_loss: 0.4259 - val_mean_absolute_error: 0.5733
Epoch 56/450
1/1 [==============================] - 0s 395ms/step - loss: 1.0295 - mean_absolute_error: 0.6824 - val_loss: 0.4972 - val_mean_absolute_error: 0.6166
Epoch 57/450
1/1 [===========

1/1 [==============================] - 0s 451ms/step - loss: 1.0034 - mean_absolute_error: 0.7384 - val_loss: 0.2771 - val_mean_absolute_error: 0.4173
Epoch 102/450
1/1 [==============================] - 0s 403ms/step - loss: 0.7504 - mean_absolute_error: 0.6606 - val_loss: 0.3905 - val_mean_absolute_error: 0.5121
Epoch 103/450
1/1 [==============================] - 0s 398ms/step - loss: 0.8701 - mean_absolute_error: 0.6657 - val_loss: 0.3256 - val_mean_absolute_error: 0.4639
Epoch 104/450
1/1 [==============================] - 0s 417ms/step - loss: 0.7412 - mean_absolute_error: 0.6123 - val_loss: 0.3621 - val_mean_absolute_error: 0.4943
Epoch 105/450
1/1 [==============================] - 0s 413ms/step - loss: 0.4774 - mean_absolute_error: 0.5423 - val_loss: 0.3792 - val_mean_absolute_error: 0.5206
Epoch 106/450
1/1 [==============================] - 0s 378ms/step - loss: 0.5341 - mean_absolute_error: 0.6028 - val_loss: 0.3293 - val_mean_absolute_error: 0.4637
Epoch 107/450
1/1 [=====

Epoch 151/450
1/1 [==============================] - 0s 470ms/step - loss: 0.4379 - mean_absolute_error: 0.4986 - val_loss: 0.3378 - val_mean_absolute_error: 0.4682
Epoch 152/450
1/1 [==============================] - 0s 477ms/step - loss: 0.5437 - mean_absolute_error: 0.5480 - val_loss: 0.2580 - val_mean_absolute_error: 0.4041
Epoch 153/450
1/1 [==============================] - 0s 451ms/step - loss: 0.6758 - mean_absolute_error: 0.5994 - val_loss: 0.2981 - val_mean_absolute_error: 0.4548
Epoch 154/450
1/1 [==============================] - 0s 429ms/step - loss: 0.5741 - mean_absolute_error: 0.5096 - val_loss: 0.2451 - val_mean_absolute_error: 0.4180
Epoch 155/450
1/1 [==============================] - 0s 406ms/step - loss: 0.4493 - mean_absolute_error: 0.4607 - val_loss: 0.3329 - val_mean_absolute_error: 0.4833
Epoch 156/450
1/1 [==============================] - 0s 410ms/step - loss: 0.7671 - mean_absolute_error: 0.6427 - val_loss: 0.2963 - val_mean_absolute_error: 0.4582
Epoch 157/

Epoch 201/450
1/1 [==============================] - 1s 501ms/step - loss: 0.2635 - mean_absolute_error: 0.4015 - val_loss: 0.2989 - val_mean_absolute_error: 0.4574
Epoch 202/450
1/1 [==============================] - 0s 392ms/step - loss: 0.4729 - mean_absolute_error: 0.4635 - val_loss: 0.2636 - val_mean_absolute_error: 0.4413
Epoch 203/450
1/1 [==============================] - 0s 446ms/step - loss: 0.2762 - mean_absolute_error: 0.3864 - val_loss: 0.2183 - val_mean_absolute_error: 0.3828
Epoch 204/450
1/1 [==============================] - 0s 406ms/step - loss: 0.2800 - mean_absolute_error: 0.3791 - val_loss: 0.1796 - val_mean_absolute_error: 0.3379
Epoch 205/450
1/1 [==============================] - 0s 385ms/step - loss: 0.2597 - mean_absolute_error: 0.3838 - val_loss: 0.2329 - val_mean_absolute_error: 0.3858
Epoch 206/450
1/1 [==============================] - 0s 417ms/step - loss: 0.2877 - mean_absolute_error: 0.3822 - val_loss: 0.2024 - val_mean_absolute_error: 0.3478
Epoch 207/

Epoch 251/450
1/1 [==============================] - 0s 415ms/step - loss: 0.2461 - mean_absolute_error: 0.3646 - val_loss: 0.1761 - val_mean_absolute_error: 0.3170
Epoch 252/450
1/1 [==============================] - 0s 418ms/step - loss: 0.2581 - mean_absolute_error: 0.3726 - val_loss: 0.1604 - val_mean_absolute_error: 0.3116
Epoch 253/450
1/1 [==============================] - 0s 444ms/step - loss: 0.2180 - mean_absolute_error: 0.3547 - val_loss: 0.1873 - val_mean_absolute_error: 0.3645
Epoch 254/450
1/1 [==============================] - 0s 463ms/step - loss: 0.3812 - mean_absolute_error: 0.4016 - val_loss: 0.1887 - val_mean_absolute_error: 0.3367
Epoch 255/450
1/1 [==============================] - 0s 380ms/step - loss: 0.1907 - mean_absolute_error: 0.3397 - val_loss: 0.1590 - val_mean_absolute_error: 0.3129
Epoch 256/450
1/1 [==============================] - 0s 409ms/step - loss: 0.2448 - mean_absolute_error: 0.3575 - val_loss: 0.1617 - val_mean_absolute_error: 0.3173
Epoch 257/

Epoch 301/450
1/1 [==============================] - 0s 444ms/step - loss: 0.1206 - mean_absolute_error: 0.2458 - val_loss: 0.1776 - val_mean_absolute_error: 0.3433
Epoch 302/450
1/1 [==============================] - 0s 445ms/step - loss: 0.2773 - mean_absolute_error: 0.2945 - val_loss: 0.1752 - val_mean_absolute_error: 0.3397
Epoch 303/450
1/1 [==============================] - 0s 388ms/step - loss: 0.1374 - mean_absolute_error: 0.2695 - val_loss: 0.1591 - val_mean_absolute_error: 0.3086
Epoch 304/450
1/1 [==============================] - 0s 399ms/step - loss: 0.1756 - mean_absolute_error: 0.2994 - val_loss: 0.1599 - val_mean_absolute_error: 0.3090
Epoch 305/450
1/1 [==============================] - 0s 487ms/step - loss: 0.1176 - mean_absolute_error: 0.2434 - val_loss: 0.1311 - val_mean_absolute_error: 0.2810
Epoch 306/450
1/1 [==============================] - 0s 408ms/step - loss: 0.1443 - mean_absolute_error: 0.2773 - val_loss: 0.1406 - val_mean_absolute_error: 0.2884
Epoch 307/

Epoch 351/450
1/1 [==============================] - 0s 406ms/step - loss: 0.3149 - mean_absolute_error: 0.3085 - val_loss: 0.1377 - val_mean_absolute_error: 0.2800
Epoch 352/450
1/1 [==============================] - 0s 496ms/step - loss: 0.1616 - mean_absolute_error: 0.2782 - val_loss: 0.1099 - val_mean_absolute_error: 0.2635
Epoch 353/450
1/1 [==============================] - 0s 414ms/step - loss: 0.1180 - mean_absolute_error: 0.2472 - val_loss: 0.1657 - val_mean_absolute_error: 0.3090
Epoch 354/450
1/1 [==============================] - 0s 466ms/step - loss: 0.1118 - mean_absolute_error: 0.2451 - val_loss: 0.1482 - val_mean_absolute_error: 0.2971
Epoch 355/450
1/1 [==============================] - 0s 451ms/step - loss: 0.2449 - mean_absolute_error: 0.2930 - val_loss: 0.1569 - val_mean_absolute_error: 0.2769
Epoch 356/450
1/1 [==============================] - 0s 430ms/step - loss: 0.2540 - mean_absolute_error: 0.2833 - val_loss: 0.1475 - val_mean_absolute_error: 0.2938
Epoch 357/

Epoch 401/450
1/1 [==============================] - 0s 479ms/step - loss: 0.1555 - mean_absolute_error: 0.2880 - val_loss: 0.1716 - val_mean_absolute_error: 0.3283
Epoch 402/450
1/1 [==============================] - 0s 388ms/step - loss: 0.1766 - mean_absolute_error: 0.2950 - val_loss: 0.1336 - val_mean_absolute_error: 0.2729
Epoch 403/450
1/1 [==============================] - 0s 395ms/step - loss: 0.1509 - mean_absolute_error: 0.2727 - val_loss: 0.1298 - val_mean_absolute_error: 0.2700
Epoch 404/450
1/1 [==============================] - 0s 435ms/step - loss: 0.1206 - mean_absolute_error: 0.2430 - val_loss: 0.1270 - val_mean_absolute_error: 0.2666
Epoch 405/450
1/1 [==============================] - 0s 468ms/step - loss: 0.1373 - mean_absolute_error: 0.2622 - val_loss: 0.1434 - val_mean_absolute_error: 0.2939
Epoch 406/450
1/1 [==============================] - 0s 412ms/step - loss: 0.1216 - mean_absolute_error: 0.2397 - val_loss: 0.1846 - val_mean_absolute_error: 0.3335
Epoch 407/

1/1 [==============================] - 0s 138ms/step - loss: 0.1449 - mean_absolute_error: 0.2927
val performance = [0.14487820863723755, 0.29267632961273193]
test performance =  [0.14974543452262878, 0.2796904146671295]


In [82]:
plot_history(history)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [83]:
multi_window.plot2(multi_lstm_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [84]:
multi_window.plot2(multi_lstm_model, plot_col=5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**CNN model**

In [85]:
CONV_WIDTH = 7
CONV_LAYER_NO = 1
multi_conv_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, CONV_WIDTH, features]
    tf.keras.layers.Lambda(lambda x: x[:, -(CONV_WIDTH*CONV_LAYER_NO-CONV_LAYER_NO+1):, :]),
] + [
    # Shape => [batch, 1, conv_units]
    tf.keras.layers.Conv1D(1024, activation='relu', kernel_size=(CONV_WIDTH)) for i in range(CONV_LAYER_NO)
] + [
    # Shape => [batch, 1,  out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [86]:
MAX_EPOCHS = 400
history = compile_and_fit(multi_conv_model, multi_window)

#IPython.display.clear_output()

multi_val_performance['Conv'] = multi_conv_model.evaluate(multi_window.val)
multi_performance['Conv'] = multi_conv_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['Conv'])
print('test performance = ', multi_performance['Conv'])

Epoch 1/400
1/1 [==============================] - 1s 701ms/step - loss: 1.6610 - mean_absolute_error: 0.9230 - val_loss: 0.3446 - val_mean_absolute_error: 0.4977
Epoch 2/400
1/1 [==============================] - 0s 276ms/step - loss: 1.1783 - mean_absolute_error: 0.8261 - val_loss: 0.3010 - val_mean_absolute_error: 0.4357
Epoch 3/400
1/1 [==============================] - 0s 223ms/step - loss: 1.1009 - mean_absolute_error: 0.7717 - val_loss: 0.3059 - val_mean_absolute_error: 0.4089
Epoch 4/400
1/1 [==============================] - 0s 247ms/step - loss: 1.1522 - mean_absolute_error: 0.7228 - val_loss: 0.2926 - val_mean_absolute_error: 0.3931
Epoch 5/400
1/1 [==============================] - 0s 255ms/step - loss: 0.5507 - mean_absolute_error: 0.5397 - val_loss: 0.4890 - val_mean_absolute_error: 0.6037
Epoch 6/400
1/1 [==============================] - 0s 206ms/step - loss: 0.8108 - mean_absolute_error: 0.6602 - val_loss: 0.5748 - val_mean_absolute_error: 0.6357
Epoch 7/400
1/1 [=====

1/1 [==============================] - 0s 229ms/step - loss: 0.2013 - mean_absolute_error: 0.2644 - val_loss: 0.1149 - val_mean_absolute_error: 0.2386
Epoch 52/400
1/1 [==============================] - 0s 221ms/step - loss: 0.1741 - mean_absolute_error: 0.2824 - val_loss: 0.1023 - val_mean_absolute_error: 0.2369
Epoch 53/400
1/1 [==============================] - 0s 199ms/step - loss: 0.3328 - mean_absolute_error: 0.2798 - val_loss: 0.1024 - val_mean_absolute_error: 0.2337
Epoch 54/400
1/1 [==============================] - 0s 239ms/step - loss: 0.1269 - mean_absolute_error: 0.2516 - val_loss: 0.0977 - val_mean_absolute_error: 0.2382
Epoch 55/400
1/1 [==============================] - 0s 231ms/step - loss: 0.1708 - mean_absolute_error: 0.2566 - val_loss: 0.1046 - val_mean_absolute_error: 0.2398
Epoch 56/400
1/1 [==============================] - 0s 260ms/step - loss: 0.1076 - mean_absolute_error: 0.2315 - val_loss: 0.1173 - val_mean_absolute_error: 0.2586
Epoch 57/400
1/1 [===========

1/1 [==============================] - 0s 221ms/step - loss: 0.1360 - mean_absolute_error: 0.2398 - val_loss: 0.0893 - val_mean_absolute_error: 0.2308
Epoch 102/400
1/1 [==============================] - 0s 234ms/step - loss: 0.1184 - mean_absolute_error: 0.2410 - val_loss: 0.0975 - val_mean_absolute_error: 0.2477
Epoch 103/400
1/1 [==============================] - 0s 251ms/step - loss: 0.1172 - mean_absolute_error: 0.2468 - val_loss: 0.1083 - val_mean_absolute_error: 0.2534
Epoch 104/400
1/1 [==============================] - 0s 232ms/step - loss: 0.1166 - mean_absolute_error: 0.2467 - val_loss: 0.0957 - val_mean_absolute_error: 0.2342
Epoch 105/400
1/1 [==============================] - 0s 247ms/step - loss: 0.1150 - mean_absolute_error: 0.2259 - val_loss: 0.1109 - val_mean_absolute_error: 0.2415
Epoch 106/400
1/1 [==============================] - 0s 225ms/step - loss: 0.1623 - mean_absolute_error: 0.2369 - val_loss: 0.0796 - val_mean_absolute_error: 0.2168
Epoch 107/400
1/1 [=====

Epoch 151/400
1/1 [==============================] - 0s 205ms/step - loss: 0.2082 - mean_absolute_error: 0.2426 - val_loss: 0.1094 - val_mean_absolute_error: 0.2550
Epoch 152/400
1/1 [==============================] - 0s 233ms/step - loss: 0.1083 - mean_absolute_error: 0.2165 - val_loss: 0.0882 - val_mean_absolute_error: 0.2420
Epoch 153/400
1/1 [==============================] - 0s 231ms/step - loss: 0.1329 - mean_absolute_error: 0.2414 - val_loss: 0.0828 - val_mean_absolute_error: 0.2285
Epoch 154/400
1/1 [==============================] - 0s 253ms/step - loss: 0.1721 - mean_absolute_error: 0.2279 - val_loss: 0.0824 - val_mean_absolute_error: 0.2164
Epoch 155/400
1/1 [==============================] - 0s 224ms/step - loss: 0.2275 - mean_absolute_error: 0.2215 - val_loss: 0.0964 - val_mean_absolute_error: 0.2350
Epoch 156/400
1/1 [==============================] - 0s 251ms/step - loss: 0.1206 - mean_absolute_error: 0.2353 - val_loss: 0.0795 - val_mean_absolute_error: 0.2181
Epoch 157/

Epoch 201/400
1/1 [==============================] - 0s 247ms/step - loss: 0.1038 - mean_absolute_error: 0.2239 - val_loss: 0.1099 - val_mean_absolute_error: 0.2548
Epoch 202/400
1/1 [==============================] - 0s 233ms/step - loss: 0.1106 - mean_absolute_error: 0.2381 - val_loss: 0.1335 - val_mean_absolute_error: 0.2829
Epoch 203/400
1/1 [==============================] - 0s 242ms/step - loss: 0.2463 - mean_absolute_error: 0.2643 - val_loss: 0.1291 - val_mean_absolute_error: 0.2738
Epoch 204/400
1/1 [==============================] - 0s 266ms/step - loss: 0.0996 - mean_absolute_error: 0.2284 - val_loss: 0.1467 - val_mean_absolute_error: 0.2948
Epoch 205/400
1/1 [==============================] - 0s 226ms/step - loss: 0.1417 - mean_absolute_error: 0.2398 - val_loss: 0.1505 - val_mean_absolute_error: 0.3093
Epoch 206/400
1/1 [==============================] - 0s 254ms/step - loss: 0.2828 - mean_absolute_error: 0.2526 - val_loss: 0.1128 - val_mean_absolute_error: 0.2648
Epoch 207/

Epoch 251/400
1/1 [==============================] - 0s 195ms/step - loss: 0.2409 - mean_absolute_error: 0.2492 - val_loss: 0.1147 - val_mean_absolute_error: 0.2600
Epoch 252/400
1/1 [==============================] - 0s 243ms/step - loss: 0.1031 - mean_absolute_error: 0.2260 - val_loss: 0.1228 - val_mean_absolute_error: 0.2827
Epoch 253/400
1/1 [==============================] - 0s 263ms/step - loss: 0.1034 - mean_absolute_error: 0.2204 - val_loss: 0.1333 - val_mean_absolute_error: 0.2801
Epoch 254/400
1/1 [==============================] - 0s 262ms/step - loss: 0.1639 - mean_absolute_error: 0.2298 - val_loss: 0.1163 - val_mean_absolute_error: 0.2637
Epoch 255/400
1/1 [==============================] - 0s 243ms/step - loss: 0.0816 - mean_absolute_error: 0.2098 - val_loss: 0.0989 - val_mean_absolute_error: 0.2342
Epoch 256/400
1/1 [==============================] - 0s 225ms/step - loss: 0.0877 - mean_absolute_error: 0.2083 - val_loss: 0.1510 - val_mean_absolute_error: 0.2929
Epoch 257/

Epoch 301/400
1/1 [==============================] - 0s 206ms/step - loss: 0.1410 - mean_absolute_error: 0.2390 - val_loss: 0.1101 - val_mean_absolute_error: 0.2563
Epoch 302/400
1/1 [==============================] - 0s 220ms/step - loss: 0.0626 - mean_absolute_error: 0.1726 - val_loss: 0.1364 - val_mean_absolute_error: 0.2689
Epoch 303/400
1/1 [==============================] - 0s 201ms/step - loss: 0.1631 - mean_absolute_error: 0.2325 - val_loss: 0.1184 - val_mean_absolute_error: 0.2557
Epoch 304/400
1/1 [==============================] - 0s 226ms/step - loss: 0.1490 - mean_absolute_error: 0.2472 - val_loss: 0.1278 - val_mean_absolute_error: 0.2764
Epoch 305/400
1/1 [==============================] - 0s 202ms/step - loss: 0.1200 - mean_absolute_error: 0.2243 - val_loss: 0.1523 - val_mean_absolute_error: 0.3006
Epoch 306/400
1/1 [==============================] - 0s 217ms/step - loss: 0.0934 - mean_absolute_error: 0.2156 - val_loss: 0.1436 - val_mean_absolute_error: 0.3116
Epoch 307/

Epoch 351/400
1/1 [==============================] - 0s 270ms/step - loss: 0.1034 - mean_absolute_error: 0.2197 - val_loss: 0.1557 - val_mean_absolute_error: 0.2967
Epoch 352/400
1/1 [==============================] - 0s 230ms/step - loss: 0.0797 - mean_absolute_error: 0.2012 - val_loss: 0.1478 - val_mean_absolute_error: 0.2875
Epoch 353/400
1/1 [==============================] - 0s 230ms/step - loss: 0.1228 - mean_absolute_error: 0.2457 - val_loss: 0.1242 - val_mean_absolute_error: 0.2766
Epoch 354/400
1/1 [==============================] - 0s 231ms/step - loss: 0.0920 - mean_absolute_error: 0.2064 - val_loss: 0.1250 - val_mean_absolute_error: 0.2801
Epoch 355/400
1/1 [==============================] - 0s 235ms/step - loss: 0.0922 - mean_absolute_error: 0.2145 - val_loss: 0.0891 - val_mean_absolute_error: 0.2290
Epoch 356/400
1/1 [==============================] - 0s 222ms/step - loss: 0.0859 - mean_absolute_error: 0.2005 - val_loss: 0.1241 - val_mean_absolute_error: 0.2722
Epoch 357/

1/1 [==============================] - 0s 117ms/step - loss: 0.1318 - mean_absolute_error: 0.2651
val performance = [0.1318170428276062, 0.2650766968727112]
test performance =  [0.10286352038383484, 0.2282380312681198]


In [87]:
multi_val_performance['Conv'] = multi_conv_model.evaluate(multi_window.val)
multi_performance['Conv'] = multi_conv_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['Conv'])
print('test performance = ', multi_performance['Conv'])

1/1 [==============================] - 0s 125ms/step - loss: 0.1122 - mean_absolute_error: 0.2637
val performance = [0.11217811703681946, 0.2636820077896118]
test performance =  [0.19208411872386932, 0.26795732975006104]


In [88]:
plot_history(history)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [89]:
multi_window.plot2(multi_conv_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**performance**

In [90]:
x = np.arange(len(multi_performance))
width = 0.3


metric_name = 'mean_absolute_error'
metric_index = multi_conv_model.metrics_names.index('mean_absolute_error')
val_mae = [v[metric_index] for v in multi_val_performance.values()]
test_mae = [v[metric_index] for v in multi_performance.values()]

plt.figure()
plt.bar(x - 0.17, val_mae, width, label='Validation')
plt.bar(x + 0.17, test_mae, width, label='Test')
plt.xticks(ticks=x, labels=multi_performance.keys(),
           rotation=45)
plt.ylabel(f'MAE (average over all times and outputs)')
_ = plt.legend()
plt.show()

/home/cjinw/work/tf-2/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  # Remove the CWD from sys.path while we load stuff.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Autoregressive model**

In [91]:
class FeedBack(tf.keras.Model):
  def __init__(self, units, out_steps):
    super().__init__()
    self.out_steps = out_steps
    self.units = units
    self.lstm_cell = tf.keras.layers.LSTMCell(units)
    # Also wrap the LSTMCell in an RNN to simplify the `warmup` method.
    self.lstm_rnn = tf.keras.layers.RNN(self.lstm_cell, return_state=True)
    self.dense = tf.keras.layers.Dense(num_features)
    
feedback_model = FeedBack(units=32, out_steps=OUT_STEPS)

def warmup(self, inputs):
  # inputs.shape => (batch, time, features)
  # x.shape => (batch, lstm_units)
  x, *state = self.lstm_rnn(inputs)
  #print('x =', x)

  # predictions.shape => (batch, features)
  prediction = self.dense(x)
  return prediction, state

FeedBack.warmup = warmup

prediction, state = feedback_model.warmup(multi_window.example[0])
prediction.shape

def call(self, inputs, training=None):
  # Use a TensorArray to capture dynamically unrolled outputs.
  predictions = []
  # Initialize the lstm state
  prediction, state = self.warmup(inputs)

  # Insert the first prediction
  predictions.append(prediction)

  # Run the rest of the prediction steps
  for n in range(1, self.out_steps):
    # Use the last prediction as input.
    x = prediction
    # Execute one lstm step.
    x, state = self.lstm_cell(x, states=state,
                              training=training)
    # Convert the lstm output to a prediction.
    prediction = self.dense(x)
    # Add the prediction to the output
    predictions.append(prediction)

  # predictions.shape => (time, batch, features)
  predictions = tf.stack(predictions)
  # predictions.shape => (batch, time, features)
  predictions = tf.transpose(predictions, [1, 0, 2])
  predictions = tf.keras.layers.Lambda(lambda x: x[:, :, out_features[0]:(out_features[0]+len(out_features))])(predictions)
  return predictions

FeedBack.call = call

print('Output shape (batch, time, features): ', feedback_model(multi_window.example[0]).shape)

Output shape (batch, time, features):  (128, 72, 1)


In [92]:
history = compile_and_fit(feedback_model, multi_window)

#IPython.display.clear_output()

multi_val_performance['AR LSTM'] = feedback_model.evaluate(multi_window.val)
multi_performance['AR LSTM'] = feedback_model.evaluate(multi_window.test, verbose=0)

print('val performance =', multi_val_performance['AR LSTM'])
print('test performance = ', multi_performance['AR LSTM'])

Epoch 1/400
1/1 [==============================] - 5s 5s/step - loss: 2.2005 - mean_absolute_error: 1.0864 - val_loss: 0.3607 - val_mean_absolute_error: 0.5170
Epoch 2/400
1/1 [==============================] - 0s 312ms/step - loss: 1.5340 - mean_absolute_error: 0.8171 - val_loss: 0.4039 - val_mean_absolute_error: 0.5594
Epoch 3/400
1/1 [==============================] - 0s 303ms/step - loss: 1.5059 - mean_absolute_error: 0.9181 - val_loss: 0.3707 - val_mean_absolute_error: 0.5299
Epoch 4/400
1/1 [==============================] - 0s 325ms/step - loss: 1.6183 - mean_absolute_error: 0.9597 - val_loss: 0.3627 - val_mean_absolute_error: 0.5077
Epoch 5/400
1/1 [==============================] - 0s 308ms/step - loss: 1.5586 - mean_absolute_error: 0.9073 - val_loss: 0.4512 - val_mean_absolute_error: 0.5680
Epoch 6/400
1/1 [==============================] - 0s 289ms/step - loss: 1.4157 - mean_absolute_error: 0.7988 - val_loss: 0.3714 - val_mean_absolute_error: 0.5093
Epoch 7/400
1/1 [========

1/1 [==============================] - 0s 277ms/step - loss: 0.3153 - mean_absolute_error: 0.4162 - val_loss: 0.3022 - val_mean_absolute_error: 0.4335
Epoch 52/400
1/1 [==============================] - 0s 298ms/step - loss: 0.5010 - mean_absolute_error: 0.5317 - val_loss: 0.3220 - val_mean_absolute_error: 0.4592
Epoch 53/400
1/1 [==============================] - 0s 288ms/step - loss: 0.5764 - mean_absolute_error: 0.5162 - val_loss: 0.2471 - val_mean_absolute_error: 0.3938
Epoch 54/400
1/1 [==============================] - 0s 293ms/step - loss: 0.3270 - mean_absolute_error: 0.4301 - val_loss: 0.3008 - val_mean_absolute_error: 0.4463
Epoch 55/400
1/1 [==============================] - 0s 287ms/step - loss: 0.2331 - mean_absolute_error: 0.3669 - val_loss: 0.2696 - val_mean_absolute_error: 0.4099
Epoch 56/400
1/1 [==============================] - 0s 289ms/step - loss: 0.4097 - mean_absolute_error: 0.4375 - val_loss: 0.2242 - val_mean_absolute_error: 0.3846
Epoch 57/400
1/1 [===========

1/1 [==============================] - 0s 302ms/step - loss: 0.2364 - mean_absolute_error: 0.3461 - val_loss: 0.1923 - val_mean_absolute_error: 0.3434
Epoch 102/400
1/1 [==============================] - 0s 281ms/step - loss: 0.3285 - mean_absolute_error: 0.3343 - val_loss: 0.1873 - val_mean_absolute_error: 0.3347
Epoch 103/400
1/1 [==============================] - 0s 262ms/step - loss: 0.1953 - mean_absolute_error: 0.3089 - val_loss: 0.1957 - val_mean_absolute_error: 0.3318
Epoch 104/400
1/1 [==============================] - 0s 325ms/step - loss: 0.2126 - mean_absolute_error: 0.3027 - val_loss: 0.2202 - val_mean_absolute_error: 0.3758
Epoch 105/400
1/1 [==============================] - 0s 293ms/step - loss: 0.1891 - mean_absolute_error: 0.3063 - val_loss: 0.1845 - val_mean_absolute_error: 0.3316
Epoch 106/400
1/1 [==============================] - 0s 308ms/step - loss: 0.2871 - mean_absolute_error: 0.3089 - val_loss: 0.1755 - val_mean_absolute_error: 0.3202
Epoch 107/400
1/1 [=====

Epoch 151/400
1/1 [==============================] - 0s 316ms/step - loss: 0.1391 - mean_absolute_error: 0.2533 - val_loss: 0.1587 - val_mean_absolute_error: 0.3034
Epoch 152/400
1/1 [==============================] - 0s 290ms/step - loss: 0.1494 - mean_absolute_error: 0.2730 - val_loss: 0.1736 - val_mean_absolute_error: 0.3216
Epoch 153/400
1/1 [==============================] - 0s 306ms/step - loss: 0.1398 - mean_absolute_error: 0.2601 - val_loss: 0.1459 - val_mean_absolute_error: 0.3025
Epoch 154/400
1/1 [==============================] - 0s 317ms/step - loss: 0.1836 - mean_absolute_error: 0.2974 - val_loss: 0.1537 - val_mean_absolute_error: 0.2989
Epoch 155/400
1/1 [==============================] - 0s 302ms/step - loss: 0.2894 - mean_absolute_error: 0.2860 - val_loss: 0.1298 - val_mean_absolute_error: 0.2731
Epoch 156/400
1/1 [==============================] - 0s 304ms/step - loss: 0.2773 - mean_absolute_error: 0.2961 - val_loss: 0.1991 - val_mean_absolute_error: 0.3331
Epoch 157/

Epoch 201/400
1/1 [==============================] - 0s 316ms/step - loss: 0.1530 - mean_absolute_error: 0.2773 - val_loss: 0.1544 - val_mean_absolute_error: 0.2985
Epoch 202/400
1/1 [==============================] - 0s 317ms/step - loss: 0.1526 - mean_absolute_error: 0.2715 - val_loss: 0.1615 - val_mean_absolute_error: 0.3063
Epoch 203/400
1/1 [==============================] - 0s 286ms/step - loss: 0.3828 - mean_absolute_error: 0.3151 - val_loss: 0.1719 - val_mean_absolute_error: 0.3074
Epoch 204/400
1/1 [==============================] - 0s 293ms/step - loss: 0.1136 - mean_absolute_error: 0.2441 - val_loss: 0.1584 - val_mean_absolute_error: 0.3004
Epoch 205/400
1/1 [==============================] - 0s 326ms/step - loss: 0.1186 - mean_absolute_error: 0.2401 - val_loss: 0.1332 - val_mean_absolute_error: 0.2885
Epoch 206/400
1/1 [==============================] - 0s 278ms/step - loss: 0.1250 - mean_absolute_error: 0.2429 - val_loss: 0.1725 - val_mean_absolute_error: 0.3206
Epoch 207/

Epoch 251/400
1/1 [==============================] - 0s 339ms/step - loss: 0.2222 - mean_absolute_error: 0.2926 - val_loss: 0.1698 - val_mean_absolute_error: 0.3204
Epoch 252/400
1/1 [==============================] - 0s 288ms/step - loss: 0.1486 - mean_absolute_error: 0.2690 - val_loss: 0.1429 - val_mean_absolute_error: 0.2925
Epoch 253/400
1/1 [==============================] - 0s 318ms/step - loss: 0.1374 - mean_absolute_error: 0.2668 - val_loss: 0.1629 - val_mean_absolute_error: 0.3051
Epoch 254/400
1/1 [==============================] - 0s 332ms/step - loss: 0.1249 - mean_absolute_error: 0.2405 - val_loss: 0.1336 - val_mean_absolute_error: 0.2851
Epoch 255/400
1/1 [==============================] - 0s 307ms/step - loss: 0.1098 - mean_absolute_error: 0.2321 - val_loss: 0.1381 - val_mean_absolute_error: 0.2808
Epoch 256/400
1/1 [==============================] - 0s 321ms/step - loss: 0.1322 - mean_absolute_error: 0.2485 - val_loss: 0.1333 - val_mean_absolute_error: 0.2839
Epoch 257/

Epoch 301/400
1/1 [==============================] - 0s 314ms/step - loss: 0.0978 - mean_absolute_error: 0.2175 - val_loss: 0.1279 - val_mean_absolute_error: 0.2736
Epoch 302/400
1/1 [==============================] - 0s 311ms/step - loss: 0.1926 - mean_absolute_error: 0.2571 - val_loss: 0.1644 - val_mean_absolute_error: 0.3105
Epoch 303/400
1/1 [==============================] - 0s 299ms/step - loss: 0.3487 - mean_absolute_error: 0.3078 - val_loss: 0.1208 - val_mean_absolute_error: 0.2679
Epoch 304/400
1/1 [==============================] - 0s 302ms/step - loss: 0.1013 - mean_absolute_error: 0.2215 - val_loss: 0.1332 - val_mean_absolute_error: 0.2864
Epoch 305/400
1/1 [==============================] - 0s 293ms/step - loss: 0.2360 - mean_absolute_error: 0.2579 - val_loss: 0.1268 - val_mean_absolute_error: 0.2747
Epoch 306/400
1/1 [==============================] - 0s 313ms/step - loss: 0.2404 - mean_absolute_error: 0.2787 - val_loss: 0.1211 - val_mean_absolute_error: 0.2679
Epoch 307/

Epoch 351/400
1/1 [==============================] - 0s 338ms/step - loss: 0.1529 - mean_absolute_error: 0.2686 - val_loss: 0.1241 - val_mean_absolute_error: 0.2766
Epoch 352/400
1/1 [==============================] - 0s 319ms/step - loss: 0.2208 - mean_absolute_error: 0.2334 - val_loss: 0.1477 - val_mean_absolute_error: 0.2939
Epoch 353/400
1/1 [==============================] - 0s 294ms/step - loss: 0.2534 - mean_absolute_error: 0.2548 - val_loss: 0.1421 - val_mean_absolute_error: 0.3036
Epoch 354/400
1/1 [==============================] - 0s 276ms/step - loss: 0.4931 - mean_absolute_error: 0.2982 - val_loss: 0.1650 - val_mean_absolute_error: 0.3239
Epoch 355/400
1/1 [==============================] - 0s 280ms/step - loss: 0.1916 - mean_absolute_error: 0.2604 - val_loss: 0.1562 - val_mean_absolute_error: 0.3132
Epoch 356/400
1/1 [==============================] - 0s 301ms/step - loss: 0.1211 - mean_absolute_error: 0.2420 - val_loss: 0.1791 - val_mean_absolute_error: 0.3371
Epoch 357/

1/1 [==============================] - 0s 113ms/step - loss: 0.1206 - mean_absolute_error: 0.2735
val performance = [0.12057333439588547, 0.27349454164505005]
test performance =  [0.20588694512844086, 0.2787778377532959]


In [93]:
plot_history(history)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [94]:
multi_window.plot2(feedback_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**performance**

In [95]:
x = np.arange(len(multi_performance))
width = 0.3


metric_name = 'mean_absolute_error'
metric_index = multi_conv_model.metrics_names.index('mean_absolute_error')
val_mae = [v[metric_index] for v in multi_val_performance.values()]
test_mae = [v[metric_index] for v in multi_performance.values()]

plt.figure()
plt.bar(x - 0.17, val_mae, width, label='Validation')
plt.bar(x + 0.17, test_mae, width, label='Test')
plt.xticks(ticks=x, labels=multi_performance.keys(),
           rotation=45)
plt.ylabel(f'MAE (average over all times and outputs)')
_ = plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …